<h1 id="tocheading">TABLE OF CONTENTS</h1>
<div id="toc"></div>

**Updates to the table of contents are periodic, but run the cell below to first start or force an update.**

In [ ]:
macro javascript_str(s) display("text/javascript", s); end

javascript"""
$.getScript('make_table_of_contents.js')
"""

# Workflow

Here we describe how to download a set of runs from the gcloud VMs, run Alex's code to compute SVD on the dynamics, and then run Marino's code to compute the number of clusters we should look at. At that point you are good to run a browser like `LDA_browser`.

## Getting the files from gcloud

Suppose you have processes running that have generated runs in machine:directory.  First you need to download those runs.  

* If the directory is a subdirectory of superior_colliculus_mutual_inhibition, you will want those runs on git. You can therefore use git to bring any new runs over. For each machine:directory combination, use the shell (not Julia) script 

    ./pull_results.sh machine farm_id directory  [E.g., ./pull_results proanti024 C17 Farms_C17]
    
    This script goes into the gcloud VM, does a git pull, then adds any new files in the indicated directory to git, commits and pushes them, comes back to the local machine, and does a git pull.
    
* If the directory is NOT a subdirectory of superior_colliculus_mutual_inhibition, then you will want to use the shell (not Julia) script

    ./get_new.sh  machine farm_id cloud_directory local_directory  [E.g., ../get_new.sh proanti028 C19 ../Farms_C19 ../Farms_C19]
    
    This script will look for any files in the cloud directory that are NOT already in the local directory, will make a compressed tar file with them, will copy that over, and extract it locally. It then cleans up after itself, removing the tar file both remotely and locally.
    
    You might then want to make a Mini farm, a directory with small versions of the runs that is appropriate in size for putting on git. Now in Julia, after having included "results_analysis.jld", 
    
    > make_mini_farm("C19", fromdirs="../Farms_C19", todir="Mini_C19")
    
    
    
## Running Alex's code to compute the response matrix and SVD of dynamics

You must call

    include("svd_cluster.jl")
    
to use Alex's code.

* If you want the Hessian and the encoding analysis run, then call the very slow
    
    update_farm("C19", "Mini_C19")
        
* Otherwise, if you're good without those, then

   update_farm("C19", "Mini_C19", build_hessian=false, build_encoding=false)

    which is still slow but not as slow as the encoding part.
   
The main files we want are named \<farmdir\>\<farmid\>_SVD_response_matrixn[_reduced].jld . We'll use as standard n-3 and reduced, for for "C19", "Mini_C19", this is file "Mini_C19C19_SVD_response_matrix3_reduced.jld". Marino's code needs that.

At this point, you could run LDA_browser, but we haven't yet determined how many clusters to show. We use Marino's code for that.

## Running Marino's code

You must call

    include("cluster_farms.jl")l
    
to use Marino's code.

* Then within Juia call

    cluster_farms("C19", farmdir="Mini_C19")
    
    
This will produce a file "compute_clustering/MarinoCode_C19_MiniC19.jld" that has the output of the computation. For the optimal number of clusters, according to the BIC score on mixtures-of-Gaussians fits to the data,

    load("compute_clustering/MarinoCode_C19_MiniC19.jld", "ngroups")
    


# Sandlot

In [ ]:
pygui(true)
figure(300); clf();

x = -0.02:0.00001:0.02
plot(x, sqrt.(0.0001 + x.^2))

In [ ]:
if !isdefined(:plot_PA)
    include("pro_anti.jl")
end

fname = "MiniOptimized/farm_C17_Farms024_0049.jld"

mypars, extra_pars, args, params = load(fname,"mypars", "extra_pars", "args", "pars3")

new_pars = Dict(
:rule_and_delay_period  =>          1.2,
:rule_and_delay_periods =>          [1, 1.2],
:target_period          =>          0.4,
:target_periods         =>          [0.4],
:post_target_period     =>          0.0001,
:post_target_periods    =>          [0.0001],
)

mypars = merge(merge(mypars, extra_pars), new_pars)

cost, cost1s, cost2s, hP, hA, dP, dA, hBP, hBA = JJ(20, 20; verbose=true, make_dict(args, params, mypars)...)

size(hBP)

In [ ]:
Pro_targets  = extra_pars[:opto_targets][:,1]*ones(1,size(hBP,2))
Anti_targets = extra_pars[:opto_targets][:,1]*ones(1,size(hBA,2))
mean(abs.([Pro_targets Anti_targets] - [hBP hBA]))
all(hBP .> hBA)

hBA[2,:] .<= hBA[1,:]

hBA

# Example of loading a C17 run and finding its gradient and Hessian

In [ ]:
if !isdefined(:plot_PA)
    include("pro_anti.jl")
end

fname = "MiniOptimized/farm_C17_Farms024_0049.jld"

mypars, extra_pars, args, params = load(fname,"mypars", "extra_pars", "args", "pars3")

# Use this line if you want a new random seed for the trials
# extra_pars[:seedrand] = Int64(round(time()*1000))

# func1 =  (;params...) -> JJ(20, 20; # mypars[:nPro], mypars[:nAnti]; 
#    verbose=true, merge(merge(mypars, extra_pars), Dict(params))...)[1]


val, grad, hess = keyword_vgh(func1, args, params)

# results_analysis.jl -- Analyzing results from C17 farms

The functions in `C17_browser.jl` run mostly from a databse of summarized results generated by `farmload()` (defined below). The exception is the SVD analysis, which uses a file of pre-computed analysis generated by Alex.

`farmload()` expects that .jld file, corresponding to a training run, will have the following entries:

* `"args"`   a String vector containing the names of the trained parameters
* `"pars3"`  a Float64 vector, containing the value of the parameters names in `"args"`. 

    Note the funny name! In the output of `farmload()`, these will correspond to `"params"`, for historical reasons in the .jld files we expect them labeled as `"pars3"`


* `"cost"`   a goodness-of-fit measure, run on test trials, not training trials. (On the C17 farm, this was from measured 10000 test trials per condition.)
* `"tcost"`  the same goodness-of-fit measure as in `"cost"`, but evaluated on the training trials


**Future** analysis methods may well be expected to read other things from the .jld file, such as the Hessian at the pars3 values, percent binarized correect per condition, or other items that may be used for improved cost functions.  The trajectory of how the pars3 got trained may also be of interest.


## farmload()

In [ ]:
#@include_me  results_analysis.jl


if !isdefined(:plot_PA)
    @printf("Loading pro_anti.jl\n")
    include("pro_anti.jl")
end

using HDF5

####################################################################
#                                                                  
#   Define some helper functions for loading farms,
#   plotting histograms over the parameters, and managing
#   with GUI interactivity.
#
####################################################################


"""
results = farmload(farm_id; farmdir="../NewFarms", verbose=true, verbose_every=10)

Package and return a summary of results from a number of runs.

# PARAMETERS:

- filename     farm_id  (e.g., "C17"); .jld files containing this pattern will be loaded

# OPTIONAL PARAMETERS:

- farmdir      String indicating directory in which the files are found; alternatively, a vector
               of strings; runs from all of those directories will be loaded.

- verbose      If true, print a brief progress report every verbose_every files loaded

- verbose_every  Used if verbose==true (see above)

# RETURNS:

A dictionary with the keys:
    - "tcost" (vector or training costs), 
    - "cost" (vector of test costs), 
    - "dirs" (vector directories in which files  are found), 
    - "files" (vector of filenames, including paths), 
    - "qu_out" (deprecated for farm C17 and other non-pre-search farms), 
    - "params"" (nruns-by-nparams matrix of paramters), 
    - "args" (nparams-long vector of parameter names).
    - "grads" (nruns-by-nparams matrix of training cost gradients). If not available in the file, all elements will be "nothing".
    - "hessians" (nruns vector of nparams-by-nparams hessian matrices). If not available in the file, all elements will be "nothing".

Note that most values are nruns-long vectors but "args" is only nparams-long, because they are
the same names across all the runs.
"""
function farmload(farm_id; farmdir="../NewFarms", verbose=true, verbose_every=50)

    if typeof(farmdir)==String; farmdir=[farmdir]; end
    
    results = Dict(); dirs=[]; files =[]; qs=[]; tcosts=[]; costs=[]; pars=[]; hBPs=[]; hBAs=[];
    grads = []; hessians = Array{Array{Float64}}(0,1);
    n=0;
    for dd in farmdir
        for f in filter(x -> startswith(x, "farm_" * farm_id * "_"), readdir(dd * "/"))
            n += 1
            myfile = dd * "/" * f; 
            fj = jldopen(myfile); 
            if exists(fj, "qu_out"); qu_out = load(myfile, "qu_out"); 
            else;                    qu_out = [-1 -1]; 
            end
            # if exists(fj, "ftraj3"); ftraj3 = load(myfile, "ftraj3"); mygrad=ftraj3[1,end]'; myhess=ftraj3[2,end]
            # else                     
                ftraj3 = nothing;            mygrad=nothing;        myhess=nothing
            # end
            close(fj)

            args, params, traj3, cost, hBP, hBA = load(myfile, "args", "pars3", "traj3", "cost", "hBP", "hBA")

            if     !haskey(results, args);       results["args"] = args;
            elseif !all(results["args"].==args); error("Not all files have same args!"); 
            end

            files = [files ; myfile]; dirs = [dirs ; dd]
            qs = [qs ; qu_out]; tcosts = [tcosts; traj3[2,end]]; costs = [costs; cost]
            if length(pars) ==0; pars =params';  else pars  = [pars  ; params']; end
            if length(hBPs) ==0; hBPs =hBP';     else hBPs  = [hBPs  ; hBP'];    end
            if length(hBAs) ==0; hBAs =hBA';     else hBAs  = [hBAs  ; hBA'];    end
            
            if mygrad==nothing
                if length(grads)==0; grads=[mygrad];   else grads = [grads ; [mygrad]]; end
            else
                if length(grads)==0; grads=mygrad;     else grads = [grads ; mygrad];   end
            end
            if myhess==nothing
                if length(hessians)==0; hessians = [nothing]; 
                else hessians = [hessians ; [nothing]];
                end
            else
                hessians = [hessians ; [myhess]]
            end
            
            if verbose && rem(n, verbose_every)==0
                @printf("%s %g\n", myfile, tcosts[end])
            end
        end
    end

    results["dirs"]     = dirs
    results["files"]    = files
    results["qu_out"]   = qs
    results["tcost"]    = tcosts
    results["cost"]     = costs
    results["params"]   = pars
    results["grads"]    = grads
    results["hessians"] = hessians
    results["hBP"]      = hBPs
    results["hBA"]      = hBAs

    return results
end

## interactive scatterplot utilities

In [ ]:
unique([1 2 3 ; 1 2 3 ; 4 5 6], 1)

In [ ]:
#@include_me  results_analysis.jl


"""
Data structure for interactive scatterplots

`interactive_scatter()` returns one of these objects; used to manage GUI handling.

A mutlidimensional set of data points is plotted in a set of scatterplots, each of which 
shows the scatterplot of one dimension against another. Each data point is identified
by a unique string.  When the user clicks on one of the axes, the closest point to it
is identified, and if defined, a callback function is called, with the string ID that
datapoint passed as one of its parameters.

The points can be divided into different subsets of them, with each set plotted
in its own color. In addition, a series of initially invisible dots can also be 
added to the plot. If scatter_highlight() is defined as the callback function, then
these initially invidible points will become sequentially visible as the user clicks 
on the scatterplots.

The main function that is called to set things up is `interactive_scatter()`. The 
usual callback is `scatter_highlight()`. 

These functions work with a scatter_data data structure, defined here.

The data structure's fields are:

    Data::Array{Float64}     # npoints-by-ndims Array{Float64} of original data
    stringIDs::Array{String} # npoints-long vector of unique strings, each identifying the corresponding rows of Data. (E.g., the filename from which that point came.)
    I::Array{Array{Int64}}   # Vector of index vectors. Each element, for example, I[i] is a vector, whose elements in turn are integers, indicating the row numbers in Data that correspond to the "set i" points
    # --- stuff about graphics handling of the plots:
    axisHandles::Array{PyCall.PyObject}  # handles to the plotted axes
    axisDims::Array{Int64}               # naxes-by-2. Each row indicates the x- and y-axes dimension for each axis plot. E.g., if row j contains [2 3] that means that the jth scatterplot has column 2 of Data as its x-axis and column 3 of Data as its y-axis
    dotHandles::Array{PyCall.PyObject}   # naxes-by-ndots, handles to the extra, initially invisible, dots plotted
    callback::Any                        # A function that could be called after a button press. 
                                         # Function should be defined as taking two parameters, callback(str, SD::scatterdata). 
                                         # str will be one of the strings in stringIDs

"""
type scatter_data
    Data::Array{Float64}     # npoints-by-ndims Array{Float64} of data
    stringIDs::Array{String} # npoints-long vector of unique strings, identifying the corresponding rows of Data. (E.g., the filename from which that point came.)
    I::Array{Array{Int64}}   # Vector of index vectors. I[i] is a vector, containing row numbers of "set i" points
    # --- stuff about graphics handling of the plots:
    axisHandles::Array{PyCall.PyObject}  # handles to the plotted axes
    axisDims::Array{Int64}               # naxes-by-2, indicating x- and y-axes dimension for each axis plot
    dotHandles::Array{PyCall.PyObject}   # naxes-by-ndots, extra dots plotted
    callback::Any                        # A function to be called after a button press
end

"""
SD = interactive_scatters(Data, stringIDs; set_indices=nothing, 
    plot_set2=false, axisDims = [2 1 ; 3 1], user_callback=nothing,
    n_invisible_dots = 3, invisible_colors = ["c"; "b"; "m"],
    fignum = nothing, axisHandles = nothing, plot_colors = ["r"; "g"; "k"; "y" ; "m"], 
    markersize=10, marker=".")

A multidimensional set of data points is plotted in a set of scatterplots, each of which 
shows the scatterplot of one dimension against another. Each data point is identified
by a unique string.  When the user clicks on one of the axes, the closest point to it
is identified, and if defined, a callback function is called, with the string ID that
datapoint passed as one of its parameters.

The points can be divided into different subsets of them, with each set plotted
in its own color. In addition, a series of initially invisible dots can also be 
added to the plot. If scatter_highlight() is defined as the callback function, then
these initially invidible points will become sequentially visible as the user clicks 
on the scatterplots.

The main function that is called to set things up is `interactive_scatter()`. The 
usual callback is `scatter_highlight()`. 

This function, `interactive_scatter()`: Given a set of multidimensional data points, 
puts up at most two scatterplots of different dimensions
against each other. (If data has only two dimensions, only one scatterplot goes up.)
In addition, enables GUI interactivity: users can associate a callback function
with buttonclicks on the plots. Also puts up some invisible points on the plot with can later
be used by `scatter_highlight()`.  Returns a data structure with info as to what is plotted where,
meant to be used by GUI callbacks and functions such as `scatter_highlight()`.

If the user clicks on one of the plots, the plotted data point closest to the clicked point
will be identified, and if the user callback was defined, then user_callback will be called 
as user_callback(stringID, SD)

If desired, only a subset of the points in Data can be plotted; and multiple different subsets can be 
requested to be plotted, in different colors.

# PARAMETERS:

- Data          An npoints-by-ndims Array{Float64}

- stringIDs     And npoints-long vector of unique strings, each of which will be used to 
                identify the corresponding row in Data.

# OPTIONAL PARAMETERS:

- set_indices           Default is a vector with one element, which itself is 1:size(Data,1), 
                        i.e., default will plot all points in Data as "set 1" points. 
                        If passed, set_indices should be a vector of vectors; each
                        element should be a vectors of integers, each within 1:size(Data,2). 
                        The rows of Data in set_indices[i] will be plotted with color plot_colors[i].

- plot_colors           n-row Array, with row i containing the color to be used for set i.

- markersize            Size of each point in the scatterplots

- marker                Type of each point in the scatterplots

- plot_set2             If true, dots for sets 2:end are plotted, otherwise not.

- axisDims              Array of integers, Naxes-by-2 in size. Each row indicates with dimensions
                        to show as horizontal axis (first column of axisDims) and vertical axis (second column)
                        in the corresponding axis.

- user_callback         If set, function that will be called, as `user_callback(stringID, SD)` where
                        stringID is the string for the row that corresponds to the selected point.

- fignum                If optional parameters axisHandles is not passed, then this will indicate the
                        Figure number on which to plot the scatterplots.If not passed, a new figure is created.

- axisHandles           Two element vector containing PyPlot axis handles, indicating where to put up
                        the scatterplots. If not passed, the figure is cleared and two side-by-side 
                        subplots will be made.

- n_invisible_dots      Number of auxiliary, initially invisible, dots to put up

- invisible_colors      n_invisible_dots-long vector, indicating the color that each dot, when made
                        visible, should have.  The dots will be plotted with the first color on top.

# RETURNS

- SD::scatter_data      A structure, holindg information about the plot. See documentation for scatter_data   


# EXAMPLE

```jldoctest
pygui(true)
npoints    = 10
Data       = randn(npoints,2)
string_IDs = map(x -> @sprintf("%d", x), 1:npoints)
remove_all_BPs()  # delete any previous click handlers, for cleanliness

SD = interactive_scatters(Data, string_IDs, fignum=20, user_callback=scatter_highlight);
```jldoctest

""" 
function interactive_scatters(Data, stringIDs; set_indices=nothing, 
    plot_set2=false, axisDims = [2 1 ; 3 1], user_callback=nothing,
    n_invisible_dots = 3, invisible_colors = ["c"; "b"; "m"],
    fignum = nothing, axisHandles = nothing, plot_colors = ["r"; "g"; "k"; "y" ; "m"], 
    markersize=10, marker=".")

    @doc """
    scatter_event_callback(xy, r, linehandle, axhandle, SD::scatter_data)

    Internal function used by `interactive_scatters()` to enable GUI interactivity.
    This function is responsible for turning the position of the 
    selected data point into the corresponding filename, and then
    calling the callback that was registered with `interactive_scatters()` (if any was)

    Out of the entries in SD::scatter_data, this function uses axisHandles, axisDims,
    Data, stringIDs, and callback.

    """ function scatter_event_callback(xy, r, linehandle, axhandle, SD::scatter_data)
        # @printf("xy=(%g,%g)\n", xy[1], xy[2])
        idx = nothing
        # Let's go through the axes finding our axis
        for i=1:length(SD.axisHandles)
            if axhandle == SD.axisHandles[i]            
                myX = SD.axisDims[i,1]; myY = SD.axisDims[i,2]
                # and now find the index of the point at xy
                idx = find((SD.Data[:, myX].==xy[1]) .& (SD.Data[:, myY].==xy[2]))
                if length(idx)==0; 
                    warn(@sprintf("scatter_event_callback: Couldn't find point (%.3f,%.3f), returning\n", 
                        xy[1], xy[2]), bt=true); 
                    return; 
                end
                idx = idx[1]
            end
        end

        @printf("You selected the point with ID %s\n", SD.stringIDs[idx]); 
        pause(0.0001)  # just to get the above printed out

        # If there is a user callback, call it:
        if SD.callback != nothing
            SD.callback(SD.stringIDs[idx], SD)
        end        
    end 

    # ---------------  OK, now the actual interactive_scatters() function -------------
    
    # Initialize a scatter_data structure
    SD = scatter_data([], [], [], [], [], [], nothing)
    SD.callback  = user_callback
    SD.stringIDs = stringIDs
    SD.Data      = Data

    # Don't try to plot dimensions we don't have:
    axisDims[find(axisDims.>size(Data,2))] = size(Data,2)
    axisDims = unique(axisDims, 1)
    nplots = axisHandles==nothing ? size(axisDims,1) : minimum((size(axisDims, 1), length(axisHandles)))
    
    # Default is to plot data from all rows as set1
    if set_indices == nothing
        set_indices = [1:size(Data,1)]
    end
    
    if length(plot_colors) < length(set_indices)
        error("Need at least as many plot_colors as there are different groups of set_indices")
    end
    
    # Store indices in the SD structure that will be returned
    SD.I = I = set_indices

    # If we weren't given the axes, make them:
    if axisHandles == nothing
        # If we weren't given a figure, make it:
        if fignum==nothing
            fignum = figure()[:number]
        end
        # If we're making axes, clear the figure for them:
        figure(fignum); clf();
        if nplots==1
            axisHandles = [gca()]
        else
            axisHandles = [subplot(1,2,1), subplot(1,2,2)]
        end
    else
        # We were given axes, get figure number from them:
        if nplots==1; fignum = axisHandles[1][:figure][:number]
        else          
            fignum = [axisHandles[1][:figure][:number], axisHandles[2][:figure][:number]]
            if fignum[1]==fignum[2]; fignum=fignum[1]; end
        end
    end
    # Store in return structure:
    SD.axisHandles  = axisHandles
    SD.axisDims     = axisDims

    # Now plot the points:
    for i=1:length(SD.axisHandles)
        safe_axes(SD.axisHandles[i])
        # Find the rows that correspond to these axes:
        myX = SD.axisDims[i,1]; myY = SD.axisDims[i,2]
        for i=length(set_indices):-1:1
            if i==1 || plot_set2
                plot(Data[set_indices[i],myX],  Data[set_indices[i],myY], ".", color=plot_colors[i], 
                    markersize=markersize, marker=marker, linestyle="None")
            end            
        end
        title(@sprintf("Dim %d vs %d", myY, myX))
    end

    # Add the invisible dots:
    hs =Array{PyCall.PyObject}(0, n_invisible_dots)
    for i=1:length(SD.axisHandles)
        safe_axes(SD.axisHandles[i]); xpos = mean(xlim()); ypos = mean(ylim())
        myh = []
        for j=1:n_invisible_dots
            # Last one to be plotted should be first color (so it'll go on top):
            myh = [myh; plot(xpos, ypos, ".", color=invisible_colors[end-(j-1)], marker=marker)]
        end
        hs = [hs ; reshape(myh[end:-1:1], 1, n_invisible_dots)]
    end
    for h in hs; h[:set_markersize](markersize); h[:set_visible](false); end
    SD.dotHandles = hs
        
    for f in fignum
        install_nearest_point_callback(figure(f), scatter_event_callback, user_data=SD)
    end

    return SD 

end


"""
    scatter_highlight(stringID, SD::scatter_data)

A multidimensional set of data points is plotted in a set of scatterplots, each of which 
shows the scatterplot of one dimension against another. Each data point is identified
by a unique string.  When the user clicks on one of the axes, the closest point to it
is identified, and if defined, a callback function is called, with the string ID that
datapoint passed as one of its parameters.

The points can be divided into different subsets of them, with each set plotted
in its own color. In addition, a series of initially invisible dots can also be 
added to the plot. If scatter_highlight() is defined as the callback function, then
these initially invidible points will become sequentially visible as the user clicks 
on the scatterplots.

The main function that is called to set things up is `interactive_scatter()`. The 
usual callback is `scatter_highlight()`. 

This function, 'scatter_highlight()`:
Finds the row in SD.stringIDs that equals stringID, and then for each axis in SD.axisHandles,
sets the first SD.dotHandle's x,y data to the positon of the corresponding row of SD.Data; 
moves the second SD.dotHandle to where the first used to be; moves the third to where the
second used to be; and so on.

# EXAMPLE:

```jldoctest
SD = interactive_scatters(randn(10,3), map(x -> @sprintf("%d", x), 1:10), fignum=20)

scatter_highlight("1", SD)
scatter_highlight("5", SD)
scatter_highlight("9", SD)
```

"""
function scatter_highlight(stringID, SD::scatter_data)
    idx = find(SD.stringIDs .== stringID)
    if length(idx)==0; @printf("scatter_highlight: Couldn't find stringID %s, returning\n", stringID); return; end

    Data = SD.Data[idx,:]

    # Move our dots along:
    if length(SD.dotHandles) > 0            
        # The dot in column X will get the coords of the dot in column X-1:
        for i=1:length(SD.axisHandles)
            for to=size(SD.dotHandles,2):-1:2
                from = to-1
                SD.dotHandles[i,to][:set_xdata](SD.dotHandles[i,from][:get_xdata]())
                SD.dotHandles[i,to][:set_ydata](SD.dotHandles[i,from][:get_ydata]())
                SD.dotHandles[i,to][:set_visible](SD.dotHandles[i,from][:get_visible]())
            end
            # And then the dot in column 1 gets the coords of the red dot closest to the clicked point:
            myX = SD.axisDims[i,1]; myY = SD.axisDims[i,2]
            SD.dotHandles[i,1][:set_xdata](Data[myX])
            SD.dotHandles[i,1][:set_ydata](Data[myY])   
            SD.dotHandles[i,1][:set_visible](true)
        end
    end
    safe_axes(SD.axisHandles[end])    
    legend(SD.dotHandles[end,1:3], ["current", "1 ago", "2 ago"])
    pause(0.0001)
end




In [ ]:
pygui(true)
Data = randn(10,2)
remove_all_BPs()
SD = interactive_scatters(Data, map(x -> @sprintf("%d", x), 1:10), fignum=20, user_callback=scatter_highlight);

In [ ]:
res = farmload("C17"; farmdir="MiniOptimized")
res["params"][:,4] = abs.(res["params"][:,4])

In [ ]:
res2 = selectize(res, "cost < -0.0002")[2]


In [ ]:
set1_indices = [2, 5, 7]
set2_indices = [2, 4]

setdiff(set2_indices, set1_indices)

figure(11); clf();

h = plot([1,2,4], [3,4, 5], linestyle="None", marker=".", markersize=12)[1]



## histo_params(), plot_PCA(), and plot_SVD()

In [ ]:
#@include_me  results_analysis.jl



"""
    Data structure for parameter and cost histograms

`histo_params()` returns one of these objects; used to manage GUI handling.
"""
type histo_data
    names::Array{String}
    values::Array{Float64}
    axisHandles::Array{PyCall.PyObject}
    LineHandles::Array{PyCall.PyObject}
    files::Array{String} 
end




"""
HD = histo_params(args, params, tcosts, costs, files; fignum=1, nbins=10)

Histogram each parameter. params should be nentries-by-length(args) in size.
args should be a vector of strings. tcosts and costs should be nentries in length,
and represent trainig cost, and test cost, respectively. files should be a vector
of filenames.

# OPTIONAL PARAMS

- fignum   The figure in which histograms will be plotted.

- nbins    The number of bins to use in each histogram.

"""
function histo_params(args, params, tcosts, costs, files; fignum=1, nbins=10, linewidth=3)

    pygui(true)
    figure(fignum); clf();
    
    HD = histo_data([], [], [], [], [])

    nparams = size(params,2)
    nrows = ceil(nparams/3)+1

    for i=1:nparams;
        HD.axisHandles = [HD.axisHandles ; subplot(nrows,3,i)]; 
        plt[:hist](params[:,i], nbins)
        title(args[i])
        
        myrow = ceil(i/3)
        if myrow < (nrows+1)/2;     axisHeightChange(0.8, lock="t")
        elseif myrow > (nrows+1)/2; axisHeightChange(0.8, lock="b")
        else                        axisHeightChange(0.8, lock="c")
        end    
    end

    HD.axisHandles = [HD.axisHandles ; subplot(nrows, 2, nrows*2-1)]; axisHeightChange(0.8, lock="b"); 
    axisMove(0, -0.025); plt[:hist](tcosts*1000, nbins); title("training cost*1000")

    HD.axisHandles = [HD.axisHandles ; subplot(nrows, 2, nrows*2)];   axisHeightChange(0.8, lock="b"); 
    axisMove(0, -0.025); plt[:hist](costs*1000, nbins); title("test cost*1000")

    for ax in HD.axisHandles
        safe_axes(ax)
        h = plot([0 0 0 ; 0 0 0], [ylim()[1] ; ylim()[2]]*ones(1,3), visible=false, linewidth=linewidth)
        h[1][:set_color]("m"); h[2][:set_color]("b"); h[3][:set_color]("c")
        HD.LineHandles = [HD.LineHandles ; reshape(h[end:-1:1], 1, 3)]
    end
    
    args   = [args ; ["train cost" ; "test cost"]]
    params = [params tcosts*1000 costs*1000]
    
    HD.names  = args
    HD.values = params
    HD.files  = files
    
    return HD
end


"""
    histo_params(res; threshold=-0.0001, further_params...)

Wrapper that calls the other histo_params method, after first selecting for
only  runs that have a test cost less than threshold.  further_params are passed
on to the other histo_params method.

- threshold             training costs below this value are considered "successful" (red dots), 
                        above it are "unsuccessful" (blue dots)

- cost_choice           String, used to indicate which cost will be used for thresholding. It 
                        must be either "cost", indicating the testing cost, or "tcost", the training cost. 

- fignum                The figure in which histograms will be plotted.

- nbins                 The number of bins to use in each histogram.


"""
function histo_params(res; threshold=-0.0001, cost_choice="cost", further_params...)
    args   = res["args"]
    params = res["params"]
    tcost  = res["tcost"]
    cost   = res["cost"]
    files  = res["files"]
    
    if cost_choice=="cost"
        I = find(cost.<threshold)
    elseif cost_choice=="tcost"
        I = find(tcost.<threshold)
    else
        error("cost_choice MUST be one of \"tcost\" or \"cost\"")
    end
    
    return histo_params(args, params[I,:], tcost[I], cost[I], files[I,:]; Dict(further_params)...)
end


"""
    histo_highlight(filename, HD::histo_data)

Assuming that histo_params was called, and returned the HD that is passed to this function,
this function will put up a vertical bar at the values corresponding to the run indicated
by filename.

Will put up to three bars, blue for the most recent one asked for; green for one call ago
to this function; and magenta for the time the function was called two calls ago. That allows
keeping track of a sequence of requested values.

This function can be used for GUI interactivity-- if a button click selects a run, the
corresponding filename can then be used with this function to highlight that run in the
histograms.

# EXAMPLE:

```jldoctest
res = farmload("C17", farmdir="MiniFarms")

HD = histo_params(res)
I = find(res["cost].<-0.0001)  # to match what histo_params shows

histo_highlight(res["files"][I[1]], HD)
histo_highlight(res["files"][I[2]], HD)
histo_highlight(res["files"][I[3]], HD)
```
"""
function histo_highlight(filename, HD::histo_data)
    idx = find(HD.files .== filename)
    if length(idx)==0; @printf("histo_highlight: Couldn't find filename %s, returning\n", filename); return; end
    
    for i=1:length(HD.names)
        for to=size(HD.LineHandles,2):-1:2
            from=to-1
            HD.LineHandles[i,to][:set_xdata](HD.LineHandles[i,from][:get_xdata]())
            HD.LineHandles[i,to][:set_ydata](HD.LineHandles[i,from][:get_ydata]())
            HD.LineHandles[i,to][:set_visible](HD.LineHandles[i,from][:get_visible]())
        end        
        HD.LineHandles[i,1][:set_xdata]([HD.values[idx,i], HD.values[idx,i]])
        HD.LineHandles[i,1][:set_visible](true)
    end
end


"""
Data structure for PCA scatterplots

`plot_PCA()` returns one of these objects; used to manage GUI handling.
"""
type PCAplot_data
    # --- stuff about the data that is plotted:
    mu::Array{Float64}   # This vector of means is subtracted from a params vector
    sd::Array{Float64}   # Which are then ./ by this sd vector to get the whitened params vector
    V::Array{Float64}    # matrix of principal components, one per column
    C::Array{Float64}    # covariance matrix after whitening
    D::Array{Float64}    # Vector of eigenvalues
    I::Array{Int64}      # index into "successful" runs
    nI::Array{Int64}     # index into "unsuccessful"runs
    Vparams::Array{Float64}  # nruns-by-nparams vector of runs, in PCA space
    files::Array{String} # nruns-long vector of corresponding filenames
    # --- stuff about graphics handling of the plots:
    axisHandles::Array{PyCall.PyObject}  # handles to the plotted axes
    axisPCs::Array{Int64}    # naxes-by-2, indicating x- and y-axes PC # for each axis plot
    dotHandles::Array{PyCall.PyObject} # naxes-by-ndots, extra dots plotted
    BP::PyCall.PyObject # the kbMonitorModule.kb_monitor that will keep track of button presses
    callback::Any   # The function to be called after a button press
end





"""
    PCA_highlight(filename, PC::PCAplot_data)

Assuming that `PCA_plot()` was called, and returned the PC that is passed to this function,
this function will put up a colored dot, across all axes, at the values corresponding to the run 
indicated by filename. [Out of the entries in PC::PCAplot_data, this function uses files, 
Vparams, axHandles, dothandles, and axisPCs]

Will put up to three dots, blue for the most recent one asked for; green for one call ago
to this function; and magenta for the time the function was called two calls ago. That allows
keeping track of a sequence of requested values.

This function can be used for GUI interactivity-- if a button click selects a run, the
corresponding filename can then be used with this function to highlight that run in the
scatterplots.

# EXAMPLE:

```jldoctest
res = farmload("C17", farmdir="MiniFarms")

PC = plot_PCA(res)
I = find(res["tcost].<-0.0002)  # to match what histo_params shows

PCA_highlight(res["files"][I[1]], PC)
PCA_highlight(res["files"][I[2]], PC)
PCA_highlight(res["files"][I[3]], PC)
```
"""
function PCA_highlight(fname, PC::PCAplot_data)
    idx = find(PC.files .== fname)
    if length(idx)==0; @printf("PCA_highlight: Couldn't find filename %s in the PCAplot_data structure, returning\n", fname); return; end

    Vparams = PC.Vparams[idx,:]

    # Move our non-red dots along:
    if length(PC.dotHandles) > 0            
        # The dot in column X will get the coords of the dot in column X-1:
        for i=1:length(PC.axisHandles)
            for to=size(PC.dotHandles,2):-1:2
                from = to-1
                PC.dotHandles[i,to][:set_xdata](PC.dotHandles[i,from][:get_xdata]())
                PC.dotHandles[i,to][:set_ydata](PC.dotHandles[i,from][:get_ydata]())
                PC.dotHandles[i,to][:set_visible](PC.dotHandles[i,from][:get_visible]())
            end
            # And then the dot in column 1 gets the coords of the red dot closest to the clicked point:
            myX = PC.axisPCs[i,1]; myY = PC.axisPCs[i,2]
            PC.dotHandles[i,1][:set_xdata](Vparams[end-(myX-1)])
            PC.dotHandles[i,1][:set_ydata](Vparams[end-(myY-1)])   
            PC.dotHandles[i,1][:set_visible](true)
        end
    end
    safe_axes(PC.axisHandles[end])    
    legend(PC.dotHandles[end,1:3], ["current", "1 ago", "2 ago"])
    pause(0.001)
end


"""
    PCA_event_callback(xy, r, linehandle, axhandle, PC::PCAplot_data)

Internal function used by `PCA_plot()` to enable GUI interactivity.
This function is responsible for turning the position of the 
selected data point into the corresponding filename, and then
calling the callback that was registered with `PCA_plot()` (if any was)

Out of the entries in PC::PCAplot_data, this function uses axisHandles, axisPCs,
Vparams, files, and callback.

"""
function PCA_event_callback(xy, r, linehandle, axhandle, PC::PCAplot_data)
    # @printf("xy=(%g,%g)\n", xy[1], xy[2])
    idx = nothing
    # Let's go through the axes finding our axis
    for i=1:length(PC.axisHandles)
        if axhandle == PC.axisHandles[i]            
            myX = PC.axisPCs[i,1]; myY = PC.axisPCs[i,2]
            # and now find the index of the point at xy
            idx = find((PC.Vparams[:, end-(myX-1)].==xy[1]) .& (PC.Vparams[:, end-(myY-1)].==xy[2]))
            if length(idx)==0; 
                @printf("PCA_event_callback: Couldn't find point (%.3f,%.3f), returning\n", xy[1], xy[2]); 
                return; 
            end
            idx = idx[1]
        end
    end
    
    @printf("You selected file %s\n", PC.files[idx]); pause(0.0001)    
    
    # If there is a user callback, call it:
    if PC.callback != nothing
        PC.callback(PC.files[idx], PC)
    end        
end 
    




"""
    PC = plot_PCA(res; threshold=-0.0002, fignum=2, pc_offset=0, plot_unsuccessful=true,
        compute_good_only=true, unsuccessful_threshold=nothing, cost_choice="cost",
        user_callback=nothing)

Computes the principal components for a matrix of parameter values across many runs, and puts
up several scatterplots of the run parameter values projected onto these principal 
components. In addition, enables GUI interactivity: users can associate a callback function
with buttonclicks on the figure.

# PARAMETERS:

- res     A dictionary, in the format of the output of `farmload()`

# OPTIONAL PARAMETERS:

- threshold             costs below this are considered "successful" runs

- unsuccessful_threshold   costs above this are considered "UNsuccessful" runs
                        This value defaults to whatever threshold is.  If the two thresholds are
                        different, farms with costs in between, in the "no man's land", are not plotted.

- plot_unsuccessful     If true, dots for unsuccessful runs are shown, otherwise not.

- compute_good_only   If true, all runs, whether successful or not, are used to compute the PCs.
                        If false, only the successful runs.

- cost_choice           String, used to indicate which cost will be used for thresholding. It 
                        must be either "cost", indicating the testing cost, or "tcost", the training cost. 

- pc_offset             Shows PCs from pc_offset+1 to pc_offset+4

- user_callback         If set, function that will be called, as `usercallback(filename, PC)` where
                        filename is the name of the run that corresponds to the selected point.

- fignum                Figure number on which to plot the PC scatterplots

""" 
function plot_PCA(res; threshold=-0.0001, fignum=2, pc_offset=0, plot_unsuccessful=true,
    compute_good_only=true, unsuccessful_threshold=nothing, cost_choice="cost",
    user_callback=nothing)

    if unsuccessful_threshold==nothing
        unsuccessful_threshold = threshold
    end

    # Initialize a PCAplot_data structure
    PC = PCAplot_data([], [], [], [], [], [], [], [], [], [], [], [], nothing, nothing)

    # Get the runs' costs and do selection on them:
    if ~((cost_choice=="tcost")  ||  (cost_choice=="cost"))
        error("cost_choice MUST be one of \"tcost\" or \"cost\"")
    end    
    mycost = res[cost_choice]; 
    PC.I  = I  = find(mycost .< threshold)
    PC.nI = nI = find(mycost .>= unsuccessful_threshold)

    if compute_good_only
        # Use the successful runs (sparams) to define PCA space
        sparams = copy(res["params"][I,:])
        for i=1:size(sparams,2)
            sparams[:,i] -= mean(sparams[:,i])
            sparams[:,i] /= std(sparams[:,i])
        end
        C = sparams'*sparams/size(sparams,1)
        D,V = eig(C)
        pv = 100*D/sum(D)
    end

    # Then z-score everybody
    params = copy(res["params"]); nruns = size(params,1)
    PC.mu = mean(params,1);
    params = params - ones(nruns,1)*PC.mu
    PC.sd = std(params,1);
    params = params ./ (ones(nruns,1)*PC.sd)

    if !compute_good_only
        C = params'*params/nruns
        D,V = eig(C)
        pv = 100*D/sum(D)
    end
    PC.V = V

    # parameters in the eigen-coords:
    PC.Vparams = Vparams = (inv(V)*params')'

    figure(fignum); clf(); 
    # ax1 = subplot(2,2,1); axisHeightChange(0.9, lock="t")
    # ax2 = subplot(2,2,2); axisMove(0.05, 0); axisHeightChange(0.9, lock="t")
    # ax3 = subplot(2,2,3); axisHeightChange(0.9, lock="b")
    # ax4 = subplot(2,2,4); axisMove(0.05, 0); axisHeightChange(0.9, lock="b")
    # PC.axisHandles = [ax1, ax2, ax3, ax4]
    # PC.axisPCs = [1 2 ; 3 2 ; 1 3 ; 3 4] + pc_offset
    ax1 = subplot(1,2,1); ax2 = subplot(1,2,2)
    PC.axisHandles = [ax1 ; ax2]
    PC.axisPCs = [2 1 ; 3 1]

    for i=1:length(PC.axisHandles)
        safe_axes(PC.axisHandles[i])
        myX = PC.axisPCs[i,1]; myY = PC.axisPCs[i,2]
        if plot_unsuccessful
            plot(Vparams[nI,end-(myX-1)],  Vparams[nI,end-(myY-1)], "g.", markersize=10)
        end
        plot(Vparams[I,end-(myX-1)],  Vparams[I,end-(myY-1)], "r.", markersize=10)
        title(@sprintf("PCA %d (%.2f%%) vs %d (%.2f%%)", myY, pv[end-(myY-1)], myX, pv[end-(myX-1)]))
    end

        
    # Add the non-red dots:
    hs =Array{PyCall.PyObject}(0, 3)
    for i=1:length(PC.axisHandles)
        safe_axes(PC.axisHandles[i]); 
        hs = [hs ; reshape([plot(0, 0, "m.") plot(0, 0, "b.") plot(0, 0, "c.")][end:-1:1], 1, 3)]
        # order of handles gets reversed so the last one plotted -- goes on top -- is first handle
    end
    for h in hs; h[:set_markersize](11); h[:set_visible](false); end
    PC.dotHandles = hs
    legend(hs[end,1:3], ["current", "1 ago", "2 ago"])
    PC.callback = user_callback
    PC.files = res["files"]
        
    # safe_axes(ax3)
    # if plot_unsuccessful; legend(["unsuccessful", "successful"])
    # else                   legend(["successful"]) 
    # end

    install_nearest_point_callback(figure(fignum), PCA_event_callback, user_data=PC)

    return PC 
end



"""
SV = plot_SVD(;threshold =-0.0002, plot_unsuccessful=false, compute_good_only=false,
        cost_choice="cost", user_callback=nothing, fignum=100)

Uses the pre-computed SVD components for a matrix of parameter values across many runs, and puts
up scatterplots of the run parameter values projected onto these SV components (columns of the U 
matrix. In addition, enables GUI interactivity: users can associate a callback function
with buttonclicks on the figure.


# OPTIONAL PARAMETERS:

- threshold             training costs below this value are considered "successful" (red dots), 
                        above it are "unsuccessful" (blue dots)

- plot_unsuccessful     If true, dots for unsuccessful runs are shown, otherwise not.

- compute_good_only     If true, only the successful runs are used to compute the SVD space

- cost_choice           String, used to indicate which cost will be used for thresholding. It 
                        must be either "cost", indicating the testing cost, or "tcost", the training cost. 

- user_callback         If set, function that will be called, as `usercallback(filename, SV)` where
                        filename is the name of the run that corresponds to the selected point.

- fignum                Figure number on which to plot the PC scatterplots


# RETURNS

- SV     A structure of type PCAplot_data. (Only its files, Vparams, axHandles, dotHandles
         axisPCs and callback entries will have assigned values.)

    
"""
function plot_SVD(;threshold =-0.0001, plot_unsuccessful=false, compute_good_only=false,
    cost_choice="cost", user_callback=nothing, fignum=100)

    # get response matrix
    response, results = load("MiniOptimizedC17_SVD_response_matrix3.jld", "response","results");
    # set up filter by nan
    nanrows = any(isnan.(response),2);

    # Get the runs' costs and do selection on them:
    if ~((cost_choice=="tcost")  ||  (cost_choice=="cost"))
        error("cost_choice MUST be one of \"tcost\" or \"cost\"")
    end    
    mycost = results[cost_choice]; 
    if !compute_good_only
        mycost = mycost[.!vec(nanrows),:];
        disp_cost = copy(mycost);
    end
    badcost = mycost .>= threshold;
    
    # if we are computing SVD only on the good farms, update nanrows and disp_cost
    if compute_good_only
        nanrows = nanrows .| badcost;
        disp_cost = mycost[.!vec(nanrows),:];
    end    

    # Filter response matrix
    r_all = response[.!vec(nanrows),:];
    m = mean(r_all,1);
    r_all = r_all - repmat(m, size(r_all,1),1);
    F = svdfact(r_all);
    u = copy(F[:U]); 
    u1 = u[:,1];
    u2 = u[:,2];
    u3 = u[:,3];

    # Make list of just good farms
    if compute_good_only
        # nanrows filter already selects for good farms
        u1good = u1;
        u2good = u2;
        u3good = u3;
    else
        # remove bad cost farms
        u1good = u1[.!vec(badcost),:];
        u2good = u2[.!vec(badcost),:];
        u3good = u3[.!vec(badcost),:];
    end
    files = results["files"];
    files = files[.!vec(nanrows),:];

    SV = PCAplot_data([], [], [], [], [], [], [], [], [], [], [], [], nothing, nothing)
    SV.files = files
    SV.Vparams = u[:,3:-1:1]  # The column vectors are expected in REVERSE order (i.e., as returned by eig(). )
    
    pygui(true)
    figure(fignum); clf();

    ax1 = subplot(1,2,1)
    if plot_unsuccessful; plot(u[:,3],u[:,1],"bo"); end
    plot(u3good, u1good, "ro")
    title("SVD U columns 3 and 1")
    ylabel("SVD Dim 1")
    xlabel("SVD Dim 3")   
    plot(0, 0, "go")[1][:set_visible](false)

    ax2 = subplot(1,2,2)
    if plot_unsuccessful; plot(u[:,2],u[:,1],"bo"); end
    plot(u2good, u1good, "ro")
    title("SVD U columns 2 and 1")
    xlabel("SVD Dim 2")   
    remove_ytick_labels()

    SV.axisHandles = [ax1 ; ax2]
    SV.axisPCs   = [3 1 ; 2 1]

    hs =Array{PyCall.PyObject}(0, 3)
    for i=1:length(SV.axisHandles)
        safe_axes(SV.axisHandles[i]); 
        hs = [hs ; reshape([plot(0, 0, "m.") plot(0, 0, "b.") plot(0, 0, "c.")][end:-1:1], 1, 3)]
        # order of handles gets reversed so the last one plotted -- goes on top -- is first handle
    end
    for h in hs; h[:set_markersize](11); h[:set_visible](false); end
    SV.dotHandles = hs
    legend(hs[end,1:3], ["current", "1 ago", "2 ago"])

    SV.callback = user_callback
    # [Out of the entries in PC::PCAplot_data, this function uses files,  Vparams, axHandles, dothandles, and axisPCs]

    install_nearest_point_callback(figure(fignum), PCA_event_callback, user_data=SV)

    return SV
end

## plot_farm() -- show individual trials from a run

In [ ]:
include("pro_anti.jl")

In [ ]:
pygui(true)
figure(300); clf();
U = randn(3,20);
V = rand(4,20)
t = (1:20)*0.15

ax1 = subplot(3,1,1); ax2 = subplot(3,1,2); ax3 = subplot(3,1,3)
plot_PA2(t, U, V, fignum=300, plot_Us=true) # ,
# ax_set=Dict("Vax"=>ax1, "Uax"=>ax2, "Dax"=>ax3)); # , ax_set = [ax1, ax2])


In [ ]:
#@include_me  results_analysis.jl


plot_farm_trials = 10    #  The number of trials to be plotted per farm run



"""
    params = plot_farm(filename; testruns=400, setup_file=nothing, fignum=3, 
        plottables = ["V", "V[1,:]-V[4,:]"], ylabels=["V", "ProR-ProL"], 
        ylims = [[-0.02, 1.02], [-1.02, 1.02]], plot_list = [1:20;],    
        hit_linestyle="-", err_linestyle="--", xlims=nothing,
        overrideDict=Dict(), further_params...)

    Plots multiple trials from a single run of a farm.

# PARAMETERS

- filename    Eithe a String, the filename of the .jld file containing the run, 
to be loaded; OR an Array{Float64} vector, containing the parameter values.
In the later case, a setup_file must be defined, to set the other parameters 
(see optional parameters below).

# OPTIONAL PARAMETERS

- setup_file  if filename is actaually a parameter vector, then this optional parameter
              should be passed as a string pointing to a .jld file that when loaded, will contain
              variables "mypars", "extra_pars", and "search_conditions" -- the names of the arguments
              corresponding to the different entries of the parameter vector will be taken to be 
              the string versions of the keys of the dictionary "search_conditions".

- testruns    Number of trials to run. Defaults to value of global variable plot_farm_trials.

- fignum      Figure to put the plot up in.

- overrideDict   A dictionary containing any model parameter values that will
              override any values loaded from the file.  For example
              `overrideDict = Dict(:sigma=>0.001)` will run with that value
              of sigma, no whater what the file said.

- plottables   A vector of strings. Each of these strings indicates something to 
            plot; the strings will be evaluated in a context where the variables "V", "U", 
            and "t" are instantiated to have the values passed to `plot_PA()`. Thus 
            plottables=["V", "V[1,:]-V[4,:]"] indicates that two axes should be used, on
            the first one V will be plotted, on the second V[1,:]-V[4,:].
                The strings can be any arbitrary Julia expression, with the condition that
            their evaluation should produce something with either nrows or ncolumns
            equal to length(t).

- ylabels   A vector of strings, should be equal in length to plottables; Each element
            here will be used as the y label for the corresponding axis.

- ylims     A vector of Any, should be in length to plottables. If an element is
            `nothing`, then allow automatic scaling of the y axes for this axis. Otherwise,
            the element should be a 2-long vector of Float64, indicating the minimum and
            the maximum for the y axis, respectively.

- xlims     Either nothing (autoscale) or a 2-long vector of Float64, to be applied
            to all axes

- hit_linestyle  A string indicating the linestyle for hit trials. E.g., "-" or "--".
               If this is passed as the empty string, "", then hits are not plotted.

- err_linestyle  A string indicating the linestyle for error trials. E.g., "-" or "--".
               If this is passed as the empty string, "", then errors are not plotted.

- further_params    Any further keyword value params are passed on to run_ntrials.
            Note that unilke entries in overrideDict, which take the highest precedence, 
            keyword-value pairs here take the lowest-precedence: any kw-val pair that also
            appears in the farm's .jld file, or in overrideDict, will be ignored in favor
            of those higher precedence instances.

# RETURNS

- params     The parameters of the farm that was plotted.

# EXAMPLE CALL

```jldoctest
plot_farm("MiniOptimized/farm_C17_Farms024_0058.jld", fignum=200, testruns=20, setup_file="Setups/setup_C21.jld", plot_list=[1:5;],
    plottables = ["V", "V[1,:]+V[4,:] - (V[2,:]+V[3,:])"], ylabels=["V", "Rule encoding"],
    ylims = [[-0.02, 1.02], nothing],
    overrideDict=Dict(:dt=>0.0025));

```jldoctest
"""
function plot_farm(filename; testruns=400, setup_file=nothing, fignum=3, 
    plottables = ["V", "V[1,:]-V[4,:]"], ylabels=["V", "ProR-ProL"], 
    ylims = [[-0.02, 1.02], [-1.02, 1.02]], plot_list = [1:20;],    
    hit_linestyle="-", err_linestyle="--", xlims=nothing,
    overrideDict=Dict(), further_params...)

    mypars=extra_pars=args=pars3=[]  # define to be available outside if block
    if typeof(filename)==String
        mypars, extra_pars, args, pars3 = load(filename, "mypars", "extra_pars", "args", "pars3")
    else
        if setup_file==nothing
            error("If filename is not a string, need a setup file that, when loaded, defines mypars, extra_pars, and search_conditions. E.g., setup_file=\"Setups/setup_C20.jld\"")
        end
        if ! (typeof(filename)<:Array{Float64})
            error("If filename is not a string, it should be a vector of Float64s")
        end
        mypars, extra_pars, search_conditions=load(setup_file, "mypars", "extra_pars", "search_conditions")
        args = map(k -> String(k), keys(search_conditions))
        pars3 = filename
        if length(args) != length(pars3)
            error("If filename is not a string, it should be a vector with length equal to the number of keys in setup_file")
        end
    end 

    pygui(true)
    figure(fignum); clf();
    
    pstrings = ["CONTROL", "DELAY OPTO", "CHOICE OPTO"]
    for period = 1:3
        these_pars = merge(mypars, extra_pars);
        these_pars = merge(these_pars, Dict(
            :opto_times=>reshape(extra_pars[:opto_periods][period,:], 1, 2),
        ))
        these_pars = merge(Dict(further_params), these_pars)
        
        # The plot_list should be the one we give it below, not whatever was in the stored parameters
        delete!(these_pars, :plot_list)
        
        nplots  = length(plottables)
        pax_set = Array{PyCall.PyObject}(nplots,1)
        aax_set = Array{PyCall.PyObject}(nplots,1)
        for i=1:nplots
            pax_set[i] = subplot(2*nplots, 3, period + 3*(i-1));
            if i<=nplots/2; axisHeightChange(0.9, lock="t")
            else            axisHeightChange(0.9, lock="c")
            end
            
            aax_set[i] = subplot(2*nplots, 3, period + 3*(nplots+i-1));
            if i<=nplots/2; axisHeightChange(0.9, lock="c")
            else            axisHeightChange(0.9, lock="b")
            end
        end

        proVs, antiVs = run_ntrials(testruns, testruns; plot_list=plot_list, 
            ax_set = [pax_set, aax_set], 
            plottables = plottables, ylabels=ylabels, ylims=ylims, xlims=xlims,
            hit_linestyle=hit_linestyle, err_linestyle=err_linestyle, 
            merge(make_dict(args, pars3, these_pars), overrideDict)...);
        hBP = length(find(proVs[1,:]  .> proVs[4,:])) /size(proVs, 2)
        hBA = length(find(antiVs[4,:] .> antiVs[1,:]))/size(antiVs,2)
        # @printf("period %d:  hBP=%.2f%%, hBA=%.2f%%\n\n", period, 100*hBP, 100*hBA)

        safe_axes(pax_set[1]); title(@sprintf("%s  PRO hits = %.2f%%", pstrings[period], 100*hBP))
        safe_axes(aax_set[1]); title(@sprintf("ANTI hits = %.2f%%", 100*hBA))
        for i=1:(nplots-1)
            safe_axes(pax_set[i]); remove_xtick_labels(); xlabel("")
            safe_axes(aax_set[i]); remove_xtick_labels(); xlabel("")
        end
        safe_axes(pax_set[end]); remove_xtick_labels(); xlabel("")
        if period > 1
            remove_ytick_labels(pax_set)
            remove_ytick_labels(aax_set)
        end

        if period==2 && typeof(filename)<:String
            safe_axes(pax_set[1])
            text(mean(xlim()), ylim()[2] + 0.35*(ylim()[2]-ylim()[1]), filename,
                fontsize=18, horizontalalignment="center")
        end
        figure(fignum)[:canvas][:draw]()
        pause(0.0001)
    end

    for a=1:length(args)
        myarg = args[a]; while length(myarg)<20; myarg=myarg*" "; end
        @printf("%s\t\t%g\n", myarg, pars3[a])
    end

    return pars3
end

In [ ]:
# include("pro_anti.jl"); close("all")

In [ ]:
plot_farm("MiniOptimized/farm_C17_Farms024_0058.jld", fignum=200, testruns=20, setup_file="Setups/setup_C21.jld", plot_list=[1:5;],
plottables = ["V", "rule"], ylabels=["V", "Rule encoding"],
ylims = [[-0.02, 1.02], nothing], xlims=[1, 1.5], selectize="Vend[1]>0",
overrideDict=Dict(:dt=>0.0025));


In [ ]:
close("all")

In [ ]:
# ps=[0.305648, 0.699232, 2.46563, -0.0951304, 0.227575, -0.0150118, 0.284325, 0.451524, 0.071832, -0.803347, 0.381372, -1.13649]
# plot_farm(ps, fignum=200, testruns=800, setup_file="Setups/setup_C20.jld")

ps=[2.85619, 1.54222, 0.608599, -0.603101, 0.939795, 0.814543, 1.4115, -1.48907, 0.658225, -2.9977, 0.341421, 0.173432]
plot_farm(ps, fignum=200, testruns=20, setup_file="Setups/setup_C21.jld", plot_list=[1:5;],
plottables = ["V", "V[1,:]+V[4,:] - (V[2,:]+V[3,:])"], ylabels=["V", "Rule encoding"],
ylims = [[-0.02, 1.02], nothing],
overrideDict=Dict(:dt=>0.0025));




In [ ]:
include("results_analysis.jl")

## make_mini_farm()   -- take a set of farms with full runs and produce summaries of them

In [ ]:
#@include_me  results_analysis.jl

"""
make_mini_farm(farmid ; fromdirs=["../Farms024", "../Farms025", "../Farms026"], 
    todir="MiniFarms")    

Takes all the  runs in directories fromdirs, (which might not be on git),
and puts a small-size sumamry of them in todir. That todir directory will
a reasonabale size for git (only MBytes compred to GBytes)

The minifarm directory can be used for browsers in lieu of the original farms, but 
note that files in it do not include Hessian information.

# PARAMETERS:

- farmid     A pattern that needs to be matched in a filename for it to be included. E.g., "C17"

# OPTIONAL PARAMETERS:

- fromdirs  Either a String, indicating a directory, or a vector of Strings, indicating multiple 
            directories to be treated together.        

- todir     A String indicating the directory where the Mini files should go to. If the
            directory did not previously exist, creates it. If the directory was not previously
            empty, does not clear it.

# EXAMPLE:

```jldoctest
make_mini_farm("C17", fromdirs=["../Farms024], todir="MiniNew")
```jldoctest

"""
function make_mini_farm(farmid; fromdirs=["../Farms024", "../Farms025", "../Farms026"], 
        todir="MiniFarms")
    
    res = farmload(farmid, verbose=true, farmdir=fromdirs)

    if ~isdir(todir); mkdir(todir); end;

    sdict = []; dirname=[]; filename=[]

    for i in 1:length(res["tcost"])    
        mypars, extra_pars, args, pars3 = load(res["files"][i], "mypars", "extra_pars", 
            "args", "pars3")
        sdict = Dict("mypars"=>mypars, "extra_pars"=>extra_pars, 
            "args"=>args, "pars3"=>pars3)
        for k in keys(res)
            if k=="tcost"
                sdict["traj3"] = [0 ; res["tcost"][i]; 0]
            elseif !(k=="args" || k=="params")
                sdict[k] = res[k][i]
            end
        end

        dirname, filename = splitdir(res["files"][i]); dirname=splitdir(dirname)[2]

        save(todir * "/" * filename[1:9]*dirname*"_"*filename[10:end], sdict)
        if rem(i, 20)==0
            @printf("Did %d/%d\n", i, length(res["tcost"]))
        end
    end
end



In [ ]:
make_mini_farm("C17", fromdirs="../C17_Optimized_B/", todir="Mini_C17_Optimized_B")

In [ ]:
res = farmload("C17", farmdir="Mini_C17_Optimized_B/")

In [ ]:
length(find(res["cost"].<-0.0002))

## make_maxi_farm()   -- take a set of farm directories and put all their runs in the same directory using non-overlapping filenames

In [ ]:
#@include_me  results_analysis.jl

"""
    make_maxi_farm(farmid, fromdirs, todir)

Takes the runs in a passed set of farm directories and copies each file dir/fname*farmid* into
newdir/fname*farmid*_dir  so that all files live in one directory but don't have names that
step on each other. The files to be copied HAVE to start with "farm_", all others are ignored

"""
function make_maxi_farm(farm_id, fromdirs, todir)
    
    if ~isdir(todir); mkdir(todir); end;
    if typeof(fromdirs)==String; fromdirs=[fromdirs]; end
        
    for d in fromdirs
        fromname = split(d, "/")
        while fromname[1]==".." || fromname[1]==""; fromname=fromname[2:end]; end
        while fromname[end]==""; fromname=fromname[1:end-1]; end
        fromname = join(fromname)
        n = length("farm_"*farm_id*"_")
        for f in filter(x -> startswith(x, "farm_" * farm_id * "_"), readdir(d))   
            toname = todir*"/farm_"*farm_id*"_"*fromname*"_"*f[n+1:end]
            @printf("cp(%s, %s)\n", d*"/"*f, toname)
            cp(d*"/"*f, toname, remove_destination=true)
        end
    end
end



In [ ]:
# make_maxi_farm("C17", ["../Farms025b_Optim", "../Farms026b_Optim", "../Farms027b_Optim"], 
    "../C17_Optimized_B")


In [ ]:
# include("pro_anti.jl")

# C17_browser.jl    User-level code script

In [ ]:
#@include_me  C17_browser.jl

#######################################################
#
#     Actual calls to the functions to run everything
#
#######################################################

if !isdefined(:histo_params)
    # error("You must call include(\"results_analysis.jl\") before calling include(\"C17_browser.jl\")")
    include("results_analysis.jl")
end

if !isdefined(:res)
    res = farmload("C17", verbose=true, farmdir="MiniOptimized")
    res["params"][:,4] = abs.(res["params"][:,4])
end

pygui(true); 
remove_all_BPs(); # Clean up any previous links between clicks on figures and callback functions
plt[:close](1); plt[:close](2); plt[:close](3); plt[:close](4); plt[:close](5)

# Carlos' favored configuration, but adjust to suit -- use capture_current_figure_configuration() 
# to see code that reproduces a configuration you like once you find it

figure(1); set_current_fig_position(1325, 41, 640, 982)   # x, y, width, height
figure(2); set_current_fig_position(645, 785, 680, 408)
figure(3); set_current_fig_position(0, 785, 641, 407)
figure(4); set_current_fig_position(3, 23, 1288, 797)
figure(5); set_current_fig_position(1338, 998, 540, 200)

figure(5)
ax1 = subplot(2,2,1); axisMove(-0.05, 0); axisWidthChange(1.1, lock="l")
ax2 = subplot(2,2,2); axisMove(0.05, 0); axisWidthChange(0.6, lock="r")
ax3 = subplot(2,1,2); axisMove(0.05, 0)
rad = kbMonitorModule.radio_buttons(ax1, ["Don't plot trials", "Plot trials (wait for it)"])
tbx = dbx = []  # define these outside the try/catch so the vars are available outside the try/catch
try 
    tbx = kbMonitorModule.text_box(ax2, "ntrials to run ", "10")
    dbx = kbMonitorModule.text_box(ax3, "Override ", "")
catch
    @printf("\nI couldn't make the ntrials to run and Override text boxes for you.\n")
    tbx = Dict(:text=>"10")
    dbx = Dict(:text=>"")
end


##########################################################
#
#   next couple of lines define cost type and threshold
#
##########################################################

cost_choice = "cost"   # "cost" is test cost, "tcost" is training cost
threshold   = -0.00025  # Below this is a "successful" run


nsucc = length(find(res[cost_choice].<threshold))
@printf("\n%d runs being plotted as successful, with cost `%s' less than %g\n\n", nsucc, cost_choice, threshold)

# Put up the histograms
HD = histo_params(res; threshold=threshold, cost_choice=cost_choice);
pause(0.001)

# The callback function that will be called after clicking on a data dot:
function highlight_all(fname, PC, SV)
    PCA_highlight(fname, PC);   # Color the selected dots in the PC plot
    PCA_highlight(fname, SV);   # Color the selected dots in the SVD plot
    histo_highlight(fname, HD)  # Color the selected bars in the histograms
    pause(0.001);   # We don't really care about the 1 ms pause; just a convenient way to flush all pending graphics commandsj
    if rad[:value_selected] == "Plot trials (wait for it)"
        if ~isnull(tryparse(Int64, tbx[:text])); 
            ntrials = parse(Int64, tbx[:text])
        else
            @printf("Couldn't parse the ntrials to plot\n")
            ntrials = 10
        end
        try
            plot_farm(fname, testruns=ntrials, fignum=4, overrideDict=eval(parse("Dict("*dbx[:text]*")"))) 
        catch e
            @printf("Couldn't plot farm, error %s\n", e)
            catch_stacktrace()
        end
    end
end

# Put up the PCA plot
PC = plot_PCA(res; threshold=threshold, cost_choice=cost_choice, 
    user_callback = (fname, Trash) -> highlight_all(fname, PC, SV),
    plot_unsuccessful=false, unsuccessful_threshold=0.0001, compute_good_only=false, fignum=2);
pause(0.001)

# Put up the SVD plot
SV = plot_SVD(threshold=threshold, cost_choice=cost_choice, 
    user_callback = (fname, Trash) -> highlight_all(fname, PC, SV),
    plot_unsuccessful=false, compute_good_only=false, fignum=3);

# Print out some instructions for the user
docstring = """

Wait for PCA and SVD plots to come up.
Then click on any dot within the PCA plot or SVD plots to
see the corresponding data in the other plots. Click on
'Don't plot trials' in figure 5 if you want to go faster,
without running trials.

The window placement fits a 15-in Macbook Pro, but adjust
at will. Once you find window positions you like, run
    capture_current_figure_configuration()
to get copy-pastable code that reproduces it.

ntrials to run is the number of trials per condition that will
be run in plot_farm() to compute %correct. Up to 20 of them
will be plotted.

Override is a dict that will override any paramater values in
the plotted trials. For example, if you write
    :sigma=>0.2, :rule_and_delay_period=>1.6
then no matter what the run's file said, if you ask for trials
to be plotted, those trials will run
with that sigma and that rule_and_delay_period.

"""

@printf("%s", docstring)


In [ ]:
function highlight_all(fname, PC, SV)
    PCA_highlight(fname, PC);   # Color the selected dots in the PC plot
    PCA_highlight(fname, SV);   # Color the selected dots in the SVD plot
    histo_highlight(fname, HD)  # Color the selected bars in the histograms
    pause(0.001);   # We don't really care about the 1 ms pause; just a convenient way to flush all pending graphics commandsj
    if rad[:value_selected] == "Plot trials (wait for it)"
        if ~isnull(tryparse(Int64, tbx[:text])); 
            ntrials = parse(Int64, tbx[:text])
        else
            @printf("Couldn't parse the ntrials to plot\n")
            ntrials = 10
        end
        try
            plot_farm(fname, testruns=ntrials, fignum=4, overrideDict=eval(parse("Dict("*dbx[:text]*")"))) 
        catch e
            @printf("Couldn't plot farm, error %s\n", e)
            catch_stacktrace()
        end
    end
end


In [ ]:
# include("pro_anti.jl")
plot_farm("MiniOptimized/farm_C17_Farms026_0193.jld", testruns=20, fignum=4)

# LDA_browser.jl    User-level code script

In [ ]:
pygui(true)
G = matread("compute_clustering/LDA_output.mat")
set_indices = [find(G["cluster_ids"].==1), find(G["cluster_ids"].==2), find(G["cluster_ids"].==3)]
SD = interactive_scatters(G["ldparams"]', G["filenames"], set_indices=set_indices, plot_set2=true,
user_callback=scatter_highlight, fignum=20, axisDims=[1 2 ; 1 2]);


In [ ]:
pygui(true)
figure(5); set_current_fig_position(1338, 736, 540, 450)

figure(5); clf();
ax1 = axes([0.01, 0.84, 0.37, 0.15])
rad = kbMonitorModule.radio_buttons(ax1, ["Don't plot trials", "Plot trials (wait for it)"])

npla = 4; axheight = 0.1; axy = 0.02; 
plax = Array{PyCall.PyObject}(npla,1); ylax = Array{PyCall.PyObject}(npla,1)
plb  = Array{PyCall.PyObject}(npla,1); ylb  = Array{PyCall.PyObject}(npla,1)
for i=1:npla
    plax[i] = axes([0.13, axy, 0.49, axheight])
    plb[i]  = kbMonitorModule.text_box(plax[i], "plottable", "")
    ylax[i] = axes([0.75, axy, 0.24, axheight])
    ylb[i]  = kbMonitorModule.text_box(ylax[i], "ylim", "")
    
    axy = axy+axheight+0.02
end

axy += 0.04
ax3 = axes([0.13, axy, 0.86, axheight])
dbx = kbMonitorModule.text_box(ax3, "Override ", "")

axy += axheight + 0.04
ax4 = axes([0.23, axy, 0.4, axheight])
tbx = kbMonitorModule.text_box(ax4, "ntrials to run ", "6")

In [ ]:
typeof(plb)

In [ ]:
pygui(true)
figure(5); set_current_fig_position(1338, 998, 540, 200)

figure(5)
ax1 = subplot(2,2,1); axisMove(-0.05, 0); axisWidthChange(1.1, lock="l")
ax2 = subplot(2,2,2); axisMove(0.05, 0); axisWidthChange(0.6, lock="r")
ax3 = subplot(2,1,2); axisMove(0.05, 0)
rad = kbMonitorModule.radio_buttons(ax1, ["Don't plot trials", "Plot trials (wait for it)"])
tbx = dbx = []  # define these outside the try/catch so the vars are available outside the try/catch
try 
    tbx = kbMonitorModule.text_box(ax2, "ntrials to run ", "6")
    dbx = kbMonitorModule.text_box(ax3, "Override ", "")
catch
    @printf("\nI couldn't make the ntrials to run and Override text boxes for you.\n")
    tbx = Dict(:text=>"10")
    dbx = Dict(:text=>"")
end


In [ ]:
capture_current_figure_configuration()

In [ ]:
heb[:value_selected]=="hit only"

In [ ]:
figure(5); clf();
# ax7 = axes([0.01, axy, 0.8, 3*axheight])
scb = kbMonitorModule.check_buttons(gca(), [""], [true])

gca()[:get_children]()[1]

In [5]:
#@include_me  LDA_browser.jl

"""
    rad, heb, plb, ylb, slb, dbx, tbx, xlb = setup_control_figure(fignum)

Configures figure fignum to be a control figure for LDA_browser.

# RETURNS:

- rad     Radio buttons controlling whether to plot trials or not

- heb     All/Hits only/Errors only buttons

- plb     Plottables boxes. Non-empty entries here will correspond to an axis
per condition when plotting trials. Text inside these boxes can be arbitrary Julia
expressions, evaluated in a context where the following variables exist:

    - t    time vector

    - V    4-by-ntimesteps matrix of V activations

    - U    4-by-ntimesteps matrix of U activations

    - rule  timesteps-long vector, equal to 0.5*(V[1,:]+V[4,:]-V[2,:]-V[3,:]) (Pro minus Anti)

    - decis timesteps-long vector, equal to V[1,:]-V4,:] (Pro_R minus Pro_L)

    - diag  timesteps-long vector, equal to 0.5*(V[1,:]+V[3,:]-V[2,:]-V[4,:]) (difference of two diagonals)

    So for example you could put in "rule.^2" if for whatever reason you wanted to see 
    the rule squared as a function of time

- ylb     Ylim boxes for the plottables

    Put in "[min, max]" if you want to enforce certain ylimits. Leave empty for auto-scaling.

- slb   selectize box. Arbitrary Julia Boolean expressions, evaluated in a context with
        the same variables as for the plottables boxes, can be put in here. Only trials for
        which the expression is true will be plotted. So, for example, you could write:

        "t0=tbin(t, 1.1); abs.(decis[t0])>0.4"

        to see only trials in which the absolute value of the decision signal at time t=1.1 
        would be greater than 0.4

- dbx     Override dictionary box. Julia Dict-style entries can go here to override
        parameter values in the model. For example, to force running with sigma=0.1 and dt=0.005,
        no matter that the particular parameters in the file to be plotted were, you would
        write:

        :sigma=>0.1, :dt=>0.005

        

- tbx     number of trials to run box

- xlb     xlims box. Write in "[min, max]" if you want particular horizontal axis limits
          for the various plots.

- npla    Number of "plottables" boxes

"""
function setup_control_figure(fignum)
    figure(fignum); clf();
    
    # rad: plot farm versus not
    ax1 = axes([0.01, 0.84, 0.37, 0.15])
    rad = kbMonitorModule.radio_buttons(ax1, ["Plot trials (wait for it)", "Don't plot trials"])

    # heb : plot all or hits only or err only
    ax0 = axes([0.5, 0.8, 0.37, 0.19])
    heb = kbMonitorModule.radio_buttons(ax0, ["all", "hit only", "err only"])

    # plb, ylb :  the content of plottables, and their y limits
    npla = 3; axheight = 0.1; axy = 0.02; 
    plax = Array{PyCall.PyObject}(npla,1); ylax = Array{PyCall.PyObject}(npla,1)
    plb  = Array{PyCall.PyObject}(npla,1); ylb  = Array{PyCall.PyObject}(npla,1)
    for i=1:npla
        plax[i] = axes([0.13, axy, 0.49, axheight])
        plb[i]  = kbMonitorModule.text_box(plax[i], "plottable", "")
        ylax[i] = axes([0.75, axy, 0.24, axheight])
        ylb[i]  = kbMonitorModule.text_box(ylax[i], "ylim", "")

        axy = axy+axheight+0.02
    end
    # Default is to plot V, rule encoding, and Pro_R - Pro_L:
    plb[end][:set_val](" V ");               ylb[end][:set_val]("[-0.02, 1.02] ")
    plb[end-1][:set_val](" rule "); ylb[end-1][:set_val]("[-1.02, 1.02] ")
    plb[end-2][:set_val](" decis "); ylb[end-2][:set_val]("[-1.02, 1.02] ")

    # Julia expression to select individual trials for plotting
    axy += 0.02
    ax6 = axes([0.13, axy, 0.86, axheight])
    slb = kbMonitorModule.text_box(ax6, "Selectize", "")
    # ax7 = axes([0.01, axy, 0.1, axheight])
    # scb = kbMonitorModule.check_buttons(ax7, "", [true])

    axy += axheight + 0.02
    ax3 = axes([0.13, axy, 0.86, axheight])
    dbx = kbMonitorModule.text_box(ax3, "Override ", "")

    axy += axheight + 0.03
    ax4 = axes([0.2, axy, 0.3, axheight])
    tbx = kbMonitorModule.text_box(ax4, "ntrials to run ", "6")

    ax5 = axes([0.68, axy, 0.3, axheight])   # working on adding xlims control
    xlb = kbMonitorModule.text_box(ax5, "xlims", "")

    return rad, heb, plb, ylb, slb, dbx, tbx, xlb, npla
end


setup_control_figure

In [16]:
#@include_me  LDA_browser.jl

#######################################################
#
#     Actual calls to the functions to run everything
#
#######################################################

using MultivariateStats
using Clustering

if !isdefined(:histo_params)
    @printf("Loading results_analysis.jl\n")
    include("results_analysis.jl")
end

function LDA_browser(farmid, farmdir, threshold=-0.0002, cost_choice="cost", nclusters=4)
# farmid  = "C17"; farmdir = "Mini_C17_Optimized_B"
# farmid  = "C19"; farmdir = "Mini_C19"
# farmid  = "C17"; farmdir = "C17_Optimized_A"


    if !isdefined(:res) || true
        res = farmload(farmid, verbose=true, farmdir=farmdir)
        res["params"][:,4] = abs.(res["params"][:,4])
    end

    pygui(true); 
    remove_all_BPs(); # Clean up any previous links between clicks on figures and callback functions
    plt[:close](1); plt[:close](2); plt[:close](3); plt[:close](4); plt[:close](5)

    # Carlos' favored configuration, but adjust to suit -- use capture_current_figure_configuration() 
    # to see code that reproduces a configuration you like once you find it

    figure(1); set_current_fig_position(1325, 41, 640, 672)   # x, y, width, height
    figure(2); set_current_fig_position(645, 785, 680, 408)
    figure(3); set_current_fig_position(0, 785, 641, 407)
    figure(4); set_current_fig_position(3, 23, 1288, 797)
    figure(5); set_current_fig_position(1338, 736, 540, 450)

    # -------------- SET UP CONTROL FIGURE --------

    rad, heb, plb, ylb, slb, dbx, tbx, xlb, npla = setup_control_figure(5)

    nsucc = length(find(res[cost_choice].<threshold))
    @printf("\n%d runs being plotted as successful, with cost `%s' less than %g\n\n", nsucc, cost_choice, threshold)

    # Put up the histograms
    HD = histo_params(res; threshold=threshold, cost_choice=cost_choice);
    pause(0.001)

    # The callback function that will be called after clicking on a data dot:
    function highlight_all(fname, SV_space, PARAM_space)
        scatter_highlight(fname, SV_space);      # Color the selected dots in the LDA plot
        scatter_highlight(fname, PARAM_space);   # Color the selected dots in the LDA plot
        histo_highlight(fname, HD)               # Color the selected bars in the histograms
        pause(0.001);   # We don't really care about the 1 ms pause; just a convenient way to flush all pending graphics commandsj

        if rad[:value_selected] == "Plot trials (wait for it)"
            if ~isnull(tryparse(Int64, tbx[:text]));               ntrials = parse(Int64, tbx[:text])
            else; @printf("Couldn't parse the ntrials to plot\n"); ntrials = 10
            end
            plottables = Array{String}(0,1); ylims = Array{Any}(0,1); ylabels = Array{String}(0,1)
            for i=npla:-1:1
                if plb[i][:text] != ""
                    plottables = [plottables ; plb[i][:text]]
                    ylabels    = [ylabels    ; plb[i][:text]]
                    if ylb[i][:text] == ""
                        ylims = [ylims ; nothing]
                    else
                        ylims = [ylims ; [eval(parse(ylb[i][:text]))]]
                    end                
                end
            end
            xlims = xlb[:text]=="" ? nothing : eval(parse(xlb[:text]))
            if heb[:value_selected]     == "all";      hstyle="-"; estyle="--";
            elseif heb[:value_selected] == "hit only"; hstyle="-"; estyle="";
            else                                       hstyle="";  estyle="--";
            end
            selectize = slb[:text]=="" ? "true" : slb[:text]
            try
                plot_farm(fname, testruns=ntrials, fignum=4, 
                    plottables=plottables, ylabels=ylabels, ylims=ylims, xlims=xlims,
                    hit_linestyle = hstyle, err_linestyle = estyle, selectize=selectize,
                    overrideDict=eval(parse("Dict("*dbx[:text]*")"))) 
            catch e
                @printf("Couldn't plot farm, error %s\n", e)
                catch_stacktrace()
            end
        end
    end


    # -----------------  Put up the LDA plot  ---------
    # We'll use the output from Marino's Matlab code to get number of clusters.
    # The rest happens here.-
    marino_file = "compute_clustering/MarinoCode_"*farmid*"_"*farmdir*".jld"
    if isfile(marino_file)
        nclusters, cost_choice_marino, threshold_marino = 
            load(marino_file, "ngroups", "cost_choice", "threshold")
        nclusters=Int64(nclusters)
        if threshold_marino != threshold || cost_choice_marino != cost_choice
            warn("Marino file not generated with requested threshold and cost choice!\n", 
            "Number of clusters may be off.\n")
        end
    else
        warn(@sprintf("Didn't find file %s with output from Marino's code, guessing nclusters=%d\n",
        marino_file, nclusters))
    end


    alexfile = farmdir*"_"*farmid*"_SVD_response_matrix3_reduced.jld"
    results, response= load(alexfile, "results", "response")   # these include the trial-averaged PSTHS Alex produced
    idx       = find((results[cost_choice].<threshold)  .&  .!any(isnan.(response),2));
    files     = results["files"][idx]
    params    = results["params"][idx,:]
    # Hack params so that sigma is always positive
    args      = load(files[1], "args")
    params[:, find(args.=="sigma")] = abs.(params[:, find(args.=="sigma")])
    # Zero-mean the responses
    response  = response[idx,:]
    response  = response - ones(size(response,1),1)*mean(response,1)

    U, S, V = svd(response)

    # We arbitrarily use the top 4 principal components:
    ncomponents=4
    D = response*V[:,1:ncomponents]
    cluster_ids = assignments(kmeans(D', nclusters, init=:kmcen)) # , display=:iter))
    set_indices = Array{Any}(nclusters)
    for i=1:nclusters; set_indices[i] = find(cluster_ids.==i); end


    # Scatterplot in singular value space
    SV_space = interactive_scatters(D, files, set_indices=set_indices, plot_set2=true, 
        fignum=3, axisDims=[1 2], 
        user_callback=(fname, Trash) -> highlight_all(fname, SV_space, PARAM_space));

    # Now do LDA in parameter space, and put up scatter plot in parameter space
    M = fit(SubspaceLDA, params', nclusters, cluster_ids) # do multiclass LDA in param space with those cluster ids
    ld_data = transform(M, params')'                         # get the parameters in the LDA projection
    PARAM_space = interactive_scatters(ld_data, files, set_indices=set_indices, plot_set2=true,
        fignum=2, axisDims = [2 1; 3 1],
        user_callback=(fname, Trash) -> highlight_all(fname, SV_space, PARAM_space));



    # Print out some instructions for the user
    docstring = """

    Wait for PCA and SVD plots to come up.
    Then click on any dot within the PCA plot or SVD plots to
    see the corresponding data in the other plots. Click on
    'Don't plot trials' in figure 5 if you want to go faster,
    without running trials.

    The window placement fits a 15-in Macbook Pro, but adjust
    at will. Once you find window positions you like, run
        capture_current_figure_configuration()
    to get copy-pastable code that reproduces it.

    ntrials to run is the number of trials per condition that will
    be run in plot_farm() to compute %correct. Up to 20 of them
    will be plotted.

    Override is a dict that will override any paramater values in
    the plotted trials. For example, if you write
        :sigma=>0.2, :rule_and_delay_period=>1.6
    then no matter what the run's file said, if you ask for trials
    to be plotted, those trials will run
    with that sigma and that rule_and_delay_period.

    """

    @printf("%s", docstring)
end

LDA_browser (generic function with 4 methods)

In [15]:
LDA_browser("C19", "Mini_C19")

Mini_C19/farm_C19_Farms_C19_proanti028_0047.jld -0.000299559
Mini_C19/farm_C19_Farms_C19_proanti028_0097.jld -0.000290237
Mini_C19/farm_C19_Farms_C19_proanti029_0004.jld -8.63529e-05
Mini_C19/farm_C19_Farms_C19_proanti029_0054.jld -4.32425e-05
Mini_C19/farm_C19_Farms_C19_proanti029_0104.jld -0.000100218
Mini_C19/farm_C19_Farms_C19_proanti030_0021.jld -2.35381e-05

118 runs being plotted as successful, with cost `cost' less than -0.0002



QPixmap::scaled: Pixmap is a null pixmap
QPixmap::scaled: Pixmap is a null pixmap
QPixmap::scaled: Pixmap is a null pixmap
QPixmap::scaled: Pixmap is a null pixmap
QPixmap::scaled: Pixmap is a null pixmap



Wait for PCA and SVD plots to come up.
Then click on any dot within the PCA plot or SVD plots to
see the corresponding data in the other plots. Click on
'Don't plot trials' in figure 5 if you want to go faster,
without running trials.

The window placement fits a 15-in Macbook Pro, but adjust
at will. Once you find window positions you like, run
    capture_current_figure_configuration()
to get copy-pastable code that reproduces it.

ntrials to run is the number of trials per condition that will
be run in plot_farm() to compute %correct. Up to 20 of them
will be plotted.

Override is a dict that will override any paramater values in
the plotted trials. For example, if you write
    :sigma=>0.2, :rule_and_delay_period=>1.6
then no matter what the run's file said, if you ask for trials
to be plotted, those trials will run
with that sigma and that rule_and_delay_period.

You selected the point with ID Mini_C19/farm_C19_Farms_C19_proanti028_0046.jld
anti_rule_strength  		0.298135
right_li

In [17]:
close("all"); remove_all_BPs()

0-element Array{Any,1}

In [ ]:
farmdir="Mini_C19"; farmid="C19"
farmdir="C17_Optimized_A"; farmid="C17"
farmdir="Mini_C17_Optimized_B"; farmid="C17"
threshold=-0.0002

G1 = load(farmdir*"_"*farmid*"_SVD_response_matrix3.jld")
u = find((G1["results"]["cost"].<threshold)  .&  .!any(isnan.(G1["response"]),2));
# u = u[[1:39 ; 41:end]]
files2 = G1["results"]["files"][u]
params = G1["results"]["params"][u,:]
sel_res = G1["response"][u,:]
sel_res = sel_res - ones(size(sel_res,1),1)*mean(sel_res,1)

U, S, V = svd(sel_res)

nclusters=5

D = sel_res*V[:,1:4]
cluster_ids = assignments(kmeans(D', nclusters, init=:kmcen, display=:iter))

set_indices = Array{Any}(nclusters)
for i=1:nclusters
    set_indices[i] = find(cluster_ids.==i)
end
set_indices

figure(100); clf()
# for i=1:length(set_indices)
#    plot(D[set_indices[i],1], D[set_indices[i],2], ".", markersize=11)
#end

function mycallback(fname, IS, LD)
    scatter_highlight(fname, IS)
    scatter_highlight(fname, LD)    
    plot_farm(fname, fignum=4, 
        plottables=["V", "rule", "decis"], ylabels = ["V", "rule", "decis"],
        ylims=[[-0.02, 1.02], [-1.02, 1.02], [-1.02, 1.02]], 
        testruns=100, plot_list=[1:6;])
end

IS = interactive_scatters(D, files2, set_indices=set_indices, plot_set2=true, fignum=100,
axisDims=[1 2], user_callback=mycallback);


M = fit(SubspaceLDA, params', nclusters, cluster_ids) # do multiclass LDA in param space with those cluster ids
ld_data = transform(M, params')'                         # get the parameters in the LDA projection

LD = interactive_scatters(ld_data, files2, set_indices=set_indices, plot_set2=true,
user_callback=(fname, Trash) -> mycallback(fname, IS, LD), 
        fignum=200, axisDims = [2 1; 3 1]);


In [ ]:
fig = figure(300); clf()
ax = Axes3D(fig)
plot_colors = ["r"; "g"; "k"; "y" ; "m"]
for i=1:length(set_indices)
    if i!=6
        plot3D(ld_data[set_indices[i],1], ld_data[set_indices[i],2], ld_data[set_indices[i],3], 
            ".", markersize=11, color=plot_colors[i])
    end
end

size(ld_data)


In [ ]:
plot_farm(files2[set_indices[8][4]], fignum=4, 
    plottables=["V", "rule", "decis"], ylabels = ["V", "rule", "decis"],
    ylims=[[-0.02, 1.02], [-1.02, 1.02], [-1.02, 1.02]], 
    testruns=100, plot_list=[1:6;])

In [ ]:
marino_file = "compute_clustering/MarinoCode_"*farmid*"_"*farmdir*".jld"
cluster_ids, bic, response2, threshold2, files2, ndim, cost_choice2,
    ngroups, results2 = load(marino_file, "idx", "bic", "response", "threshold", 
    "files", "ndim", "cost_choice", "ngroups", "results")

results2["params"]
size(cluster_ids)

In [ ]:
length(find(results2[cost_choice].<threshold))

In [ ]:
function renamer1(farmid, farmdir)
    guys = [
        "SVD_response_matrix.jld",
        "SVD_response_matrix3.jld",
        "SVD_response_matrix_reduced.jld",
        "SVD_response_matrix3_reduced.jld",
        "encoding.jld",           
        "hessians.jld",           
        "results.jld",           
    ]
    
    for i=1:length(guys)
        origname = farmdir*farmid*"_"*guys[i]
        newname  = farmdir*"_"*farmid*"_"*guys[i]
        if isfile(origname)
            @printf("git mv %s -> %s\n", origname, newname)
            run(`git mv $origname $newname`)
        else
            @printf("    -- not found: %s\n", origname)
        end        
    end    
end    

renamer1("C19", "Mini_C19")

In [ ]:
file = "tails.sh"
str = `ls $file`
run(str)
# typeof(str)

In [ ]:
plot_farm("Mini_C19/farm_C19_Farms_C19_proanti028_0001.jld", fignum=4)

In [ ]:
Array{Any}(nclusters)

In [ ]:
pygui(true); figure(1000); clf();
myrad = kbMonitorModule.radio_buttons(gca(), ["on"])


In [ ]:
# include("pro_anti.jl")

# Looking at C17 runs and preparing to optimize them

In [ ]:
res2 = farmload("C17", farmdir=["../Farms025b", "../Farms026b", "../Farms027b"])

In [ ]:
cost = res2["cost"]
tg = G["extra_pars"][:opto_targets]
n = length(res2["cost"])

hd = zeros(length(res2["cost"]))
for i=1:length(res2["cost"])
    hd[i] = mean(abs.(tg - [res2["hBP"][i,:]  res2["hBA"][i,:]]))
end
figure(100); clf();
plot(cost, hd, "b.")

In [ ]:
close("all")

In [ ]:
[false true] .& [true true]

In [ ]:
figure(100); clf();
plt[:hist](res2["cost"], -0.0003:0.00005:0.0003);
length(find(res2["cost"].<0.0002))/length(res2["cost"])
length(res2["cost"])*0.07

In [ ]:
using MultivariateStats
using Clustering
using JLD
using MAT

# old_ids  = cluster_ids
threshold = -0.0002

G1 = load("MiniOptimizedC17_SVD_response_matrix3.jld")
idx = find(G1["results"][cost_choice].<=threshold)
dynamics_data = G1["response"][idx,:]
fnames = G1["results"]["files"][idx]
params = G1["results"]["params"][idx,:]
K = kmeans(dynamics_data', 3, init=:kmcen)
cluster_ids = assignments(K)

MG = matread("compute_clustering/LDA_output.mat")
midx = find(MG[cost_choice].<=threshold)
marino_cluster_ids = MG["cluster_ids"][midx]

# Marino params and my params are not the same!!!!

use_marino = false; mycids = use_marino ? marino_cluster_ids : cluster_ids

M = fit(SubspaceLDA, params', 3, Array{Int64}(mycids))
ld_data = transform(M, params')'
# ld_data[:,2] = -ld_data[:,2]
ld_data[:,1] = -ld_data[:,1]

set_indices = [find(mycids.==1), find(mycids.==2), find(mycids.==3)]

LD = interactive_scatters(ld_data, fnames, set_indices=set_indices, plot_set2=true,
        user_callback=(fname, Trash) -> highlight_all(fname, LD, SV), fignum=20, axisDims = [1 2]);

[cluster_ids marino_cluster_ids];


In [ ]:
Array{Int64}(marino_cluster_ids)

In [ ]:
G = matread("compute_clustering/LDA_output.mat")


In [ ]:
i = find(res["files"].=="MiniOptimized/farm_C17_Farms024_0273.jld")
res["cost"][i]
length(find(res["cost"].<threshold))

In [ ]:
    res = farmload("C17", verbose=true, farmdir="MiniOptimized")
    res["params"][:,4] = abs.(res["params"][:,4])


In [ ]:
dbx[:text]

In [ ]:
    if rad[:value_selected] == "Plot trials (wait for it)"
        if ~isnull(tryparse(Int64, tbx[:text])); 
            ntrials = parse(Int64, tbx[:text])
        else
            @printf("Couldn't parse the ntrials to plot\n")
            ntrials = 10
        end
end

In [ ]:
testruns=800
mypars, extra_pars, args, pars3 = load("MiniFarms/farm_C17_Farms024_0430.jld", "mypars", "extra_pars", "args", "pars3")

cost, cost1s, cost2s, hP, hA, dP, dA, hBP, hBA = JJ(testruns, testruns; verbose=false, 
    make_dict(args, pars3, merge(merge(mypars, extra_pars)))...)

In [ ]:
[hBP hBA]

In [ ]:
ppps = plot_farm("MiniFarms/farm_C17_Farms024_0430.jld", testruns=800, fignum=10)

In [ ]:
figure(5); clf();
ax1 = subplot(2,2,1); axisMove(-0.05, 0); axisWidthChange(1.1, lock="l")
ax2 = subplot(2,2,2); axisMove(0.05, 0); axisWidthChange(0.6, lock="r")
ax3 = subplot(2,1,2); axisMove(0.05, 0)
rad = kbMonitorModule.radio_buttons(ax1, ["Don't plot trials", "Plot trials (wait for it)"])
tbx = kbMonitorModule.text_box(ax2, "ntrials to run ", "10")
dbx = kbMonitorModule.text_box(ax3, "Override ", "")


In [ ]:
safe_axes(ax1)
ylim(0, 2)

In [ ]:
res = farmload("C17", verbose=true, farmdir="MiniFarms")

res["args"]

In [ ]:
HD = histo_params(res, threshold=0.0001, fignum=1)
remove_all_BPs(); SV = plot_SVD(user_callback=
    (fname, PC) -> begin PCA_highlight(fname, PC); histo_highlight(fname, HD); end);

In [ ]:

load("MiniFarms/farm_C17_Farms024_0001.jld", "args")

In [ ]:
include("general_utils.jl")

In [ ]:
BP = __permanent_BP_store[1]
BP[:buttonlist]()

In [ ]:
H = G["ftraj3"][1,end]


In [ ]:
G = load("MiniOptimized/farm_C17_Farms024_0049.jld")

# Comparing an optimized run with a re-run using fresh trials

Here we look at how variability in the random seeds of trials affects the optimal parameters.

The cell below, extracted into standalone script `reoptimization_analysis.jl`, compares corresponding runs in the directories "MiniOptimized" and "MiniOptimized_Redux".  Files in the latter are obtained through running `optimizing_C17b.jl`, which runs through files in "MiniOptimized", gives them a new seed for random trials, and then re-optimizes, starting from the parameters that were optimal for the previous run.

The cell below takes those results and makes a plot in the first two principal components of parameter PCA space, connecting the two corresponding runs. Short connections mean similar optimal parameters.


In [ ]:
#@include_me   reoptimization_analysis.jl

if !isdefined(:plot_PA)
    @printf("Including pro_anti.jl\n")
    include("pro_anti.jl")
end

if !isdefined(:farmload)
    @printf("Including results_analysis.jl\n")
    include("results_analysis.jl")
end

if !isdefined(:reso)
    @printf("Loading MiniOptimized farms\n")
    reso = farmload("C17"; farmdir="MiniOptimized")
end

if !isdefined(:resn) || true   # forcing it to run right now so we always get latest updated files
    @printf("Loading MiniOptimized_Redux farms\n")
    resn = farmload("C17"; farmdir="MiniOptimized_Redux")
end

# The next two lines define how we threshold the optimized runs, to choose which ones go into the PCA plot
cost_choice = "cost"     # "cost" means test cost, "tcost" means training cost
threshold   = -0.0001    # Any run with a cost below this goes in

# Next we compute PCA space.
paramso = reso["params"][find(res[cost_choice].<=threshold),:]
# Whiten (mean zero, std 1):
paramso = paramso - ones(size(paramso,1),1)*mean(paramso,1)
paramso = paramso ./ (ones(size(paramso,1),1)*std(paramso,1))
# compute and diagonalize covariance matrix:
C = (paramso' * paramso)/size(paramso,1)
D, V = eig(C)
# Project old and new parameters onto eigenvectors, reverse so top PCA is first
oVparams = (inv(V)*reso["params"]')'[:,end:-1:1]
nVparams = (inv(V)*resn["params"]')'[:,end:-1:1]

figure(100); clf(); 

# Plot a red dot for all the old parameters:
u = find(res[cost_choice].<=threshold)
plot(oVparams[u,1], oVparams[u,2], "r.", markersize=11)

hg = hr = 0  # define here so vars are available outside for loop
# Now plot a green dot and a connecting line for all the new ones:
for f in readdir("MiniOptimized_Redux/")
    if startswith(f, "farm_C17") 
        iold = find(reso["files"] .== ("MiniOptimized/" * f))
        inew = find(resn["files"] .== ("MiniOptimized_Redux/" * f))
        hg = plot([oVparams[iold,1], nVparams[inew,1]], [oVparams[iold,2], nVparams[inew,2]], 
            "g.-", markersize=11)
        # Add another red dot on top of old values to keep it red
        hr = plot(oVparams[iold,1], oVparams[iold,2], "r.-", markersize=11)
    end
end

xlabel("PCA Dim 1")
ylabel("PCA Dim 2")
legend([hg[1], hr[1]], ["after re-optimizing with new test trials", "training trials"])

In [ ]:
FDversion()

# Attempt at using spectral embedding -- went nowhere, not obviously different from PCA

In [ ]:
# Looking at spectral embedding

if !isdefined(:plot_PA)
    include("pro_anti.jl")
end

if true || !isdefined(:res2)
    res=farmload("C17", farmdir="MiniOptimized/")
    res2 = selectize(res, "cost > -0.0002")[1]
end

nruns   = size(res2["params"],1)
nparams = size(res2["params"],2)
wparams = copy(res2["params"])
wparams = wparams - ones(nruns,1)*mean(wparams,1)
wparams = wparams ./ (ones(nruns, 1)*std(wparams,1))

W = zeros(nparams, nparams)
for i=1:nparams
    for j=1:nparams
        W[i,j] = sum((wparams[:,i] - wparams[:,j]).^2)
    end
end

figure(10); clf();
subplot(3,2,1); plt[:hist](W[:], 10)

offset = 0
sigma = mean(W[:])/1
W = exp.(-W/sigma)

D, V = eig(W)
VWparams = (inv(V)*wparams')'

subplot(3,2,2); plot(D)

subplot(3,2,3); plot(VWparams[:,end-(1+offset)], VWparams[:,end-offset], "r.")
xlabel(@sprintf("%d", 2+offset)); ylabel(@sprintf("%d", 1+offset))

subplot(3,2,4); plot(VWparams[:,end-(2+offset)], VWparams[:,end-offset], "r.")
xlabel(@sprintf("%d", 3+offset)); ylabel(@sprintf("%d", 1+offset))

subplot(3,2,5); plot(VWparams[:,end-(3+offset)], VWparams[:,end-(1+offset)], "r.")
xlabel(@sprintf("%d", 4+offset)); ylabel(@sprintf("%d", 2+offset))

subplot(3,2,6); plot(VWparams[:,end-(2+offset)], VWparams[:,end-(1+offset)], "r.")
xlabel(@sprintf("%d", 3+offset)); ylabel(@sprintf("%d", 2+offset))



In [ ]:
fs = readdir("MiniOptimized_Redux/")
split(fs[1])

# Checking on gradient and Hessian

In [ ]:
include("pro_anti.jl")

In [ ]:
G = load("MiniOptimized/farm_C17_Farms024_0060.jld")
mypars = G["mypars"]
extra_pars = G["extra_pars"]
args = G["args"]
params = G["pars3"]

fullparams = make_dict(args, params, merge(mypars, extra_pars))

In [ ]:
[args params]
G["cost"]

In [ ]:
H = G["ftraj3"][2,end]
D, V1 = eig(H)
# D = abs.(D)

dJ = 0.0001

sqrt.(2*dJ./sort(D))

In [ ]:
func1 =  (;params...) -> JJ(mypars[:nPro], mypars[:nAnti]; verbose=true, 
    merge(merge(mypars, extra_pars), Dict(params))...)[1]


val, grad, hess = keyword_vgh(func1, args, params)

In [ ]:
search_conditions = Dict(   # :param    default_start   search_box  bound_box
:vW     =>                   [mypars[:vW],                       [-0.5, 0.5],  [-5,   5]], 
:hW     =>                   [mypars[:hW],                       [-0.5, 0.5],  [-5,   5]],
:dW     =>                   [mypars[:dW],                       [-0.5, 0.5],  [-5,   5]],
:sW     =>                   [mypars[:sW],                       [0,    0.5],  [-5,   5]],
:sigma  =>                   [0.11,                              [0.1,  0.2],  [-2,   2]],
:constant_excitation      => [mypars[:constant_excitation],      [-1,     1],  [-30, 30]], 
:target_period_excitation => [mypars[:target_period_excitation], [0.001,0.5],  [-30  30]],
:right_light_excitation   => [mypars[:right_light_excitation],   [0.05, 0.5],  [-30, 30]],
:const_pro_bias           => [mypars[:const_pro_bias],           [-0.5, 0.5],  [-30, 30]],
:opto_strength            => [mypars[:opto_strength],            [0,      1],  [0,    1]],
:pro_rule_strength        => [mypars[:pro_rule_strength],        [0,   0.2,],  [0,   30]],
:anti_rule_strength       => [mypars[:anti_rule_strength],       [0,   0.2,],  [0,   30]],
)


bbox = Dict()
for k in keys(search_conditions)
    bbox = merge(bbox, Dict(k=>search_conditions[k][3]))
end
bbox


In [ ]:
func1 =  (;params...) -> JJ(mypars[:nPro], mypars[:nAnti]; verbose=true, 
    merge(merge(mypars, extra_pars), Dict(params))...)[1]

function internal_func(;pars...)
    fresults = func1(;wallwrap(bbox, pars)...)   # note use of bbox external to this begin...end
    if typeof(fresults)<:Tuple
        answer = fresults[1]
        further_out = fresults[2:end]
    else
        answer = fresults
    end
    return answer  # we assume that the first output of func() will always be a scalar, and that's what we return for ForwardDiff
end


In [ ]:
val, grad, hess = keyword_vgh(func1, args, params)

In [ ]:
[args params inverse_wall(bbox, args, params) vector_wrap(bbox, args, inverse_wall(bbox, args, params))]

In [ ]:
wval, wgrad, whess = keyword_vgh(internal_func, args, inverse_wall(bbox, args, params))

In [ ]:
H - whess

In [ ]:
extra_pars2 = copy(extra_pars)
# extra_pars2[:seedrand] = Int64(round(time()*1000))
# extra_pars2[:cbeta] = 0

func1 =  (;params...) -> JJ(mypars[:nPro], mypars[:nAnti]; verbose=true, 
    merge(merge(mypars, extra_pars2), Dict(params))...)[1]


val4, grad4, hess4 = keyword_vgh(func1, args, params)

In [ ]:
[G["ftraj3"][1,end] grad2 grad4]

In [ ]:
extra_pars2 = copy(extra_pars)
# extra_pars2[:seedrand] = Int64(round(time()*1000))
mypars2 = copy(mypars)
mypars2[:cbeta] = 0
extra_pars2[:cbeta] = 0

func1 =  (;params...) -> JJ(mypars[:nPro], mypars[:nAnti]; verbose=true, 
    merge(merge(mypars2, extra_pars2), Dict(params))...)[1]


val3, grad3, hess3 = keyword_vgh(func1, args, params)

In [ ]:
D, V = eig(hess3)
sort(D)

In [ ]:
extra_pars4 = copy(extra_pars)
extra_pars4[:seedrand] = Int64(round(time()*1000))
mypars2 = copy(mypars)
mypars2[:cbeta] = 0
extra_pars4[:cbeta] = 0

func1 =  (;params...) -> JJ(mypars[:nPro], mypars[:nAnti]; verbose=true, 
    merge(merge(mypars2, extra_pars4), Dict(params))...)[1]


val4, grad4, hess4 = keyword_vgh(func1, args, params)

In [ ]:
D, V = eig(hess4)
sort(D)'

# Utilities for the results database

The database is assumed to be a dictionary, with keys that are strings. One key, `"args"`, should hold a vector of strings identifying the parameters minimized. All other keys should have vectors or matrices that have nruns rows, each row corresponding to a run. They may have multiple columns, corresponding to multiple entries for that run (e.g., there is typically a `"params"` key that has, for each run, the values of all the arguments, i.e., it will have as many columns as `"args"` is long.

Some **required** keys include

* `"args"` a vector of strings, indicating the name of the trained arguments, common across runs
* `"params"`  Float64 matrix, values of the parameters corresponding to args

Some **typical** keys include

* `"tcost"`  Training cost
* `"cost"`   Test cost
* `"files"`  Name of the file in which the run is stored



In [ ]:
"""
    sres = selectize(res, selector::String)
"""
function selectize(res, selector::String)
    nruns  = size(res["params"],1)
    mykeys = setdiff(keys(res), ["args"])    
    args   = res["args"]
    
    I = zeros(nruns,1)
    for i=1:nruns
        myDict = Dict()
        for k in mykeys
            if k=="params"
                for a=1:length(args)
                    myDict[args[a]] = res[k][i,a]
                end
            else
                myDict[k] = res[k][i]
            end        
        end
        I[i] = replacer(selector, myDict)
    end
    I = find(I .!= 0)
    
    myres = copy(res)
    for k in mykeys
        myres[k] = res[k][I,:]
    end
    
    return myres, I
end

In [ ]:
testres = Dict("args"=>["t1", "t2", "p1", "p2"], "params"=>rand(6,4), "cost"=>[1;2;3;4;5;6], "z"=>[6;5;4;3;2;1])
testres["cost"][6]
selectize(testres, "p1  > 0.4")["params"][:,3]

In [ ]:
hs = plot(0, 0, "b.")

In [ ]:
BP[:clear_buttonlist]()

In [ ]:
for i in hs; i[:set_color]("y"); end
figure(2)[:canvas][:draw]()

In [ ]:
PyPlot.show()

In [ ]:
files = selectize(res, find(res["tcost"].<-0.0002))["files"]

for f in files
    plot_farm(f, testruns=200)
    
    ans = chomp(readline())
    if startswith(ans, "q") || startswith(ans, "Q")
        break
    end
end


In [ ]:
f = "farm_C17_0032.jld"
root = f[1:end-8]

## Look at individual trials

In [ ]:
args = res["args"]
I= find((res["tcost"].<-0.0002))
res["files"][I]

In [ ]:
testruns=800
mypars, extra_pars, args, pars3 = load("MiniFarms/farm_C17_Farms024_0430.jld", "mypars", "extra_pars", "args", "pars3")

cost, cost1s, cost2s, hP, hA, dP, dA, hBP, hBA = JJ(testruns, testruns; verbose=false, 
    make_dict(args, pars3, merge(merge(mypars, extra_pars)))...)

In [ ]:
pygui(true)
figure(1); clf();
figure(2); clf();

these_pars = merge(mypars, extra_pars);
these_pars = merge(these_pars, Dict(
# :opto_strength=>0.3, 
:opto_times=>reshape(extra_pars[:opto_periods][2,:], 1, 2),
# :opto_times=>["target_start-0.4" "target_start"],
# :opto_times=>["target_start" "target_end"],
:post_target_period=>0.3,
:rule_and_delay_period=>1.2,
:dt=>0.005,
))
delete!(these_pars, :plot_list)

override = Dict(
# :sigma=>0.02,
)

proVs, antiVs = run_ntrials(20, 20; plot_list=[1:20;], plot_Us=false, 
    merge(make_dict(args, pars3, these_pars), override)...);
hBP = length(find(proVs[1,:]  .> proVs[4,:])) /size(proVs, 2)
hBA = length(find(antiVs[4,:] .> antiVs[1,:]))/size(antiVs,2)
@printf("hBP=%.2f%%, hBA=%.2f%%\n", 100*hBP, 100*hBA)

figure(1); subplot(2,1,1)
title(@sprintf("PRO hits = %.2f%%", 100*hBP))
figure(2); subplot(2,1,1)
title(@sprintf("ANTI hits = %.2f%%", 100*hBA))

[args pars3]

In [ ]:
pygui(true)
figure(1); clf();
figure(2); clf();

these_pars = merge(mypars, extra_pars);
these_pars = merge(these_pars, Dict(
# :opto_strength=>0.3, 
:opto_times=>reshape(extra_pars[:opto_periods][2,:], 1, 2),
# :opto_times=>["target_start-0.4" "target_start"],
# :opto_times=>["target_start" "target_end"],
:post_target_period=>0.3,
:rule_and_delay_period=>1.2,
:dt=>0.005,
))
delete!(these_pars, :plot_list)

override = Dict(
# :sigma=>0.02,
)

proVs, antiVs = run_ntrials(100, 100; plot_list=[1:20;], plot_Us=false, 
    merge(make_dict(args, pars3, these_pars), override)...);
hBP = length(find(proVs[1,:]  .> proVs[4,:])) /size(proVs, 2)
hBA = length(find(antiVs[4,:] .> antiVs[1,:]))/size(antiVs,2)
@printf("hBP=%.2f%%, hBA=%.2f%%\n", 100*hBP, 100*hBA)

figure(1); subplot(2,1,1)
title(@sprintf("PRO hits = %.2f%%", 100*hBP))
figure(2); subplot(2,1,1)
title(@sprintf("ANTI hits = %.2f%%", 100*hBA))


In [ ]:
clf();
ax = gca();
axisHeightChange(0.5, ax=ax, lock="b")

In [ ]:
for k in keys(ax)
    print(k); print("\n")
end


In [ ]:
ax[:set_position]([0.1, 0.1, 0.1, 0.1])
b = ax[:get_position]()[:bounds]


In [ ]:
figure(2); clf()

using StatsBase

x = -0.001:0.000001:maximum(tcost)

h1 = fit(Histogram, tcost[:,1], x, closed=:right)

plot(x, [0 ; cumsum(h1.weights)]/sum(h1.weights), "b-")

xlabel("Training cost")
ylabel("fraction of runs")


In [ ]:
find(tcost.<-0.00015)

In [ ]:
# farm_id = "C11"; farmdir = "../NewFarms"

"""
    results, files, tcosts, qu_outs, sigmas, target_excites = lookem(;farm_id="C11", farmdir="../NewFarms")
"""
function lookem(;farm_id="C11", farmdir="../NewFarms")
    results = zeros(0,2); files =[]; qu_outs=[]; sigmas=[]; target_excites=[]; tcosts=[];
    for f in filter(x -> startswith(x, "farm_" * farm_id * "_"), readdir(farmdir * "/"))
        d = load(farmdir * "/" * f)
        if !haskey(d, "qu_out"); d["qu_out"] = [0 0]; end;
        traj3 = d["traj3"]; qu_out = d["qu_out"]
        @printf("%s : sigma=%.3f, q=%.1f, %g\n", f, d["pars3"][7], qu_out[1], traj3[2,end])
        results = [results ; qu_out[1] traj3[2,end]]
        files = [files ; f]
        qu_outs = [qu_outs ; d["qu_out"][1]]
        sigmas  = [sigmas ; d["pars3"][7]]
        target_excites = [target_excites ; d["pars3"][2]]
        tcosts  = [tcosts ; traj3[2,end]]
    end
    return results, files, tcosts, qu_outs, sigmas, target_excites
end

res14 = lookem(farm_id="C14")

In [ ]:
[args pars3]

In [ ]:
find(res[:,2].<-0.0002)

In [ ]:
res[res[:,2].<-0.0002,:]